### Imports and Set Flag

In [63]:
import os
import pandas as pd
import polars as pl
import numpy as np

from tqdm import tqdm


### Environment Setup ###
IS_KAGGLE = False  # Flag to switch between environments


# Progress bar - only create if running locally
N_TEST_DAYS = 21   # Number of days in test set
progress_bar = None if IS_KAGGLE else tqdm(total=N_TEST_DAYS, desc='Processing days')

Processing days:   0%|          | 0/21 [00:00<?, ?it/s]

### Environment Dependent Path Setup

In [64]:
# Setup paths based on environment
if IS_KAGGLE:
    data_path = '/kaggle/input/jane-street-real-time-market-data-forecasting'
    trained_models_path = "/kaggle/input/janestreet-models/trained_models/"
else:
    # For local testing
    import sys

    # Path to main project directory
    PROJ_DIR = os.path.dirname(os.getcwd())
    
    # Add paths to system path
    sys.path.append(os.path.join(PROJ_DIR, "jane-street-real-time-market-data-forecasting"))
    sys.path.append(os.path.join(PROJ_DIR, "training", "src", "utils"))
    
    # Set data path to our local test data
    data_path = os.path.join(os.getcwd(), "local_test_data")
    
    # Import local testing metrics
    from metrics import r2_score_weighted

import kaggle_evaluation.jane_street_inference_server

### Pre-processing

#### A global dictionary that initialises with the last seen values for the set we are about to see.

In [65]:
last_known_values = {
    (0, 'feature_00'): 0.606275,
    (0, 'feature_01'): -1.111470,
    (0, 'feature_02'): 0.957191,
    (0, 'feature_03'): 0.785588,
    (0, 'feature_04'): -1.686526,
    (0, 'feature_05'): -1.608237,
    (0, 'feature_06'): -0.264967,
    (0, 'feature_07'): -1.092005,
    (0, 'feature_08'): -0.221835,
    (0, 'feature_09'): 11.000000,
    (0, 'feature_10'): 7.000000,
    (0, 'feature_11'): 76.000000,
    (0, 'feature_12'): 0.940446,
    (0, 'feature_13'): 0.597973,
    (0, 'feature_14'): 1.052861,
    (0, 'feature_15'): -1.091804,
    (0, 'feature_16'): -0.432034,
    (0, 'feature_17'): -0.575417,
    (0, 'feature_18'): 5.115178,
    (0, 'feature_19'): -0.046730,
    (0, 'feature_20'): 0.714703,
    (0, 'feature_21'): -0.116039,
    (0, 'feature_22'): 1.779095,
    (0, 'feature_23'): 0.820886,
    (0, 'feature_24'): 0.532221,
    (0, 'feature_25'): 0.298325,
    (0, 'feature_26'): 1.257741,
    (0, 'feature_27'): 0.838528,
    (0, 'feature_28'): 0.552220,
    (0, 'feature_29'): -0.641007,
    (0, 'feature_30'): -0.839654,
    (0, 'feature_31'): -0.110346,
    (0, 'feature_32'): 1.326673,
    (0, 'feature_33'): -0.242181,
    (0, 'feature_34'): 1.296950,
    (0, 'feature_35'): 1.057961,
    (0, 'feature_36'): -0.022638,
    (0, 'feature_37'): 1.275841,
    (0, 'feature_38'): 0.977956,
    (0, 'feature_39'): -1.119926,
    (0, 'feature_40'): 0.322963,
    (0, 'feature_41'): -0.206589,
    (0, 'feature_42'): -1.528159,
    (0, 'feature_43'): -0.303905,
    (0, 'feature_44'): -1.936242,
    (0, 'feature_45'): -1.385392,
    (0, 'feature_46'): -1.339197,
    (0, 'feature_47'): -2.249058,
    (0, 'feature_48'): -0.710152,
    (0, 'feature_49'): -1.158454,
    (0, 'feature_50'): -2.041334,
    (0, 'feature_51'): -0.288202,
    (0, 'feature_52'): -1.406258,
    (0, 'feature_53'): -1.897025,
    (0, 'feature_54'): -0.838147,
    (0, 'feature_55'): -1.910240,
    (0, 'feature_56'): -2.787707,
    (0, 'feature_57'): -0.593947,
    (0, 'feature_58'): -2.743151,
    (0, 'feature_59'): -1.082671,
    (0, 'feature_60'): -1.038294,
    (0, 'feature_61'): 0.845812,
    (0, 'feature_62'): -0.343398,
    (0, 'feature_63'): -0.357768,
    (0, 'feature_64'): -0.429599,
    (0, 'feature_65'): 3.772463,
    (0, 'feature_66'): -0.884204,
    (0, 'feature_67'): 0.932611,
    (0, 'feature_68'): 1.644004,
    (0, 'feature_69'): 1.255682,
    (0, 'feature_70'): 0.679683,
    (0, 'feature_71'): 0.891455,
    (0, 'feature_72'): 1.185607,
    (0, 'feature_73'): -0.174182,
    (0, 'feature_74'): -0.271698,
    (0, 'feature_75'): 0.231436,
    (0, 'feature_76'): 0.089736,
    (0, 'feature_77'): 0.042069,
    (0, 'feature_78'): -0.045454,
    (1, 'feature_00'): 0.984515,
    (1, 'feature_01'): -1.627772,
    (1, 'feature_02'): 0.392012,
    (1, 'feature_03'): 0.467444,
    (1, 'feature_04'): -0.548078,
    (1, 'feature_05'): -1.230023,
    (1, 'feature_06'): -0.215595,
    (1, 'feature_07'): -1.405826,
    (1, 'feature_08'): -0.167783,
    (1, 'feature_09'): 11.000000,
    (1, 'feature_10'): 7.000000,
    (1, 'feature_11'): 76.000000,
    (1, 'feature_12'): 0.884171,
    (1, 'feature_13'): 2.239745,
    (1, 'feature_14'): 1.667113,
    (1, 'feature_15'): -0.607060,
    (1, 'feature_16'): -0.372970,
    (1, 'feature_17'): -0.644565,
    (1, 'feature_18'): 4.256946,
    (1, 'feature_19'): 0.032424,
    (1, 'feature_20'): 1.218869,
    (1, 'feature_21'): 0.030126,
    (1, 'feature_22'): 1.162691,
    (1, 'feature_23'): 0.546368,
    (1, 'feature_24'): 0.910786,
    (1, 'feature_25'): 1.004000,
    (1, 'feature_26'): -1.486797,
    (1, 'feature_27'): -0.283263,
    (1, 'feature_28'): 0.751058,
    (1, 'feature_29'): -0.570494,
    (1, 'feature_30'): -0.734762,
    (1, 'feature_31'): 0.044381,
    (1, 'feature_32'): 0.848509,
    (1, 'feature_33'): -0.153467,
    (1, 'feature_34'): 0.705961,
    (1, 'feature_35'): 1.020402,
    (1, 'feature_36'): 0.659197,
    (1, 'feature_37'): 1.707578,
    (1, 'feature_38'): 1.694314,
    (1, 'feature_39'): -0.726068,
    (1, 'feature_40'): 0.077357,
    (1, 'feature_41'): -0.038236,
    (1, 'feature_42'): -1.864036,
    (1, 'feature_43'): -1.207445,
    (1, 'feature_44'): -2.443120,
    (1, 'feature_45'): -0.960187,
    (1, 'feature_46'): -1.637552,
    (1, 'feature_47'): -2.357864,
    (1, 'feature_48'): -1.567318,
    (1, 'feature_49'): -1.873319,
    (1, 'feature_50'): -1.704930,
    (1, 'feature_51'): -0.451956,
    (1, 'feature_52'): -0.313873,
    (1, 'feature_53'): -2.137036,
    (1, 'feature_54'): -1.179059,
    (1, 'feature_55'): -1.890187,
    (1, 'feature_56'): -1.570160,
    (1, 'feature_57'): -1.816910,
    (1, 'feature_58'): -3.039145,
    (1, 'feature_59'): -3.267405,
    (1, 'feature_60'): -2.209712,
    (1, 'feature_61'): 0.845812,
    (1, 'feature_62'): -0.391324,
    (1, 'feature_63'): -0.331981,
    (1, 'feature_64'): -0.408544,
    (1, 'feature_65'): 4.117831,
    (1, 'feature_66'): 0.053782,
    (1, 'feature_67'): 0.898200,
    (1, 'feature_68'): 1.694648,
    (1, 'feature_69'): 1.622169,
    (1, 'feature_70'): 0.654204,
    (1, 'feature_71'): 1.099022,
    (1, 'feature_72'): 1.252282,
    (1, 'feature_73'): -0.119543,
    (1, 'feature_74'): -0.266877,
    (1, 'feature_75'): 1.723602,
    (1, 'feature_76'): 1.443849,
    (1, 'feature_77'): 0.383229,
    (1, 'feature_78'): 0.252345,
    (2, 'feature_00'): 0.062632,
    (2, 'feature_01'): -1.321548,
    (2, 'feature_02'): 0.516359,
    (2, 'feature_03'): 0.709900,
    (2, 'feature_04'): -0.775184,
    (2, 'feature_05'): -1.504639,
    (2, 'feature_06'): -0.324256,
    (2, 'feature_07'): -1.680784,
    (2, 'feature_08'): -0.041301,
    (2, 'feature_09'): 81.000000,
    (2, 'feature_10'): 2.000000,
    (2, 'feature_11'): 59.000000,
    (2, 'feature_12'): -0.114182,
    (2, 'feature_13'): 0.134089,
    (2, 'feature_14'): 0.494122,
    (2, 'feature_15'): -0.741718,
    (2, 'feature_16'): -0.484070,
    (2, 'feature_17'): -0.587165,
    (2, 'feature_18'): 0.427546,
    (2, 'feature_19'): 2.161529,
    (2, 'feature_20'): 0.795882,
    (2, 'feature_21'): -0.174311,
    (2, 'feature_22'): 0.852965,
    (2, 'feature_23'): 0.976941,
    (2, 'feature_24'): 0.638153,
    (2, 'feature_25'): -0.090669,
    (2, 'feature_26'): 0.805082,
    (2, 'feature_27'): 0.122690,
    (2, 'feature_28'): -0.307958,
    (2, 'feature_29'): -0.522356,
    (2, 'feature_30'): -0.515255,
    (2, 'feature_31'): -0.126209,
    (2, 'feature_32'): 1.067386,
    (2, 'feature_33'): -0.137791,
    (2, 'feature_34'): 1.609508,
    (2, 'feature_35'): 0.883498,
    (2, 'feature_36'): -1.292781,
    (2, 'feature_37'): 0.151407,
    (2, 'feature_38'): 0.206592,
    (2, 'feature_39'): -0.491202,
    (2, 'feature_40'): 0.358817,
    (2, 'feature_41'): -0.038733,
    (2, 'feature_42'): -2.457548,
    (2, 'feature_43'): -0.538229,
    (2, 'feature_44'): -1.589831,
    (2, 'feature_45'): -0.876616,
    (2, 'feature_46'): -1.255672,
    (2, 'feature_47'): -2.036798,
    (2, 'feature_48'): 0.224787,
    (2, 'feature_49'): -2.395313,
    (2, 'feature_50'): -0.822841,
    (2, 'feature_51'): 1.504376,
    (2, 'feature_52'): -0.024226,
    (2, 'feature_53'): -2.103225,
    (2, 'feature_54'): 0.609874,
    (2, 'feature_55'): -2.195633,
    (2, 'feature_56'): -0.359999,
    (2, 'feature_57'): -0.816383,
    (2, 'feature_58'): -2.717819,
    (2, 'feature_59'): 0.024647,
    (2, 'feature_60'): -2.772010,
    (2, 'feature_61'): 0.845812,
    (2, 'feature_62'): -0.471799,
    (2, 'feature_63'): -0.236475,
    (2, 'feature_64'): -0.345515,
    (2, 'feature_65'): 0.145268,
    (2, 'feature_66'): 1.716115,
    (2, 'feature_67'): 0.044307,
    (2, 'feature_68'): 0.179029,
    (2, 'feature_69'): 1.102428,
    (2, 'feature_70'): -0.453705,
    (2, 'feature_71'): -0.040242,
    (2, 'feature_72'): -0.118576,
    (2, 'feature_73'): -0.343052,
    (2, 'feature_74'): -0.217202,
    (2, 'feature_75'): -0.023371,
    (2, 'feature_76'): -0.048059,
    (2, 'feature_77'): -0.147135,
    (2, 'feature_78'): -0.138402,
    (3, 'feature_00'): 0.008520,
    (3, 'feature_01'): -1.595258,
    (3, 'feature_02'): 0.107887,
    (3, 'feature_03'): 0.486125,
    (3, 'feature_04'): -0.734205,
    (3, 'feature_05'): -1.382501,
    (3, 'feature_06'): -0.284784,
    (3, 'feature_07'): -1.164192,
    (3, 'feature_08'): -0.119127,
    (3, 'feature_09'): 4.000000,
    (3, 'feature_10'): 3.000000,
    (3, 'feature_11'): 11.000000,
    (3, 'feature_12'): -0.063028,
    (3, 'feature_13'): -0.038402,
    (3, 'feature_14'): -0.004865,
    (3, 'feature_15'): -0.665207,
    (3, 'feature_16'): -0.618111,
    (3, 'feature_17'): -0.661146,
    (3, 'feature_18'): -0.115859,
    (3, 'feature_19'): 1.778919,
    (3, 'feature_20'): -0.078900,
    (3, 'feature_21'): -0.093819,
    (3, 'feature_22'): 0.079027,
    (3, 'feature_23'): 0.271916,
    (3, 'feature_24'): 1.198964,
    (3, 'feature_25'): 0.667124,
    (3, 'feature_26'): -0.010588,
    (3, 'feature_27'): -0.320384,
    (3, 'feature_28'): -0.569298,
    (3, 'feature_29'): -0.580921,
    (3, 'feature_30'): -0.848901,
    (3, 'feature_31'): -0.127630,
    (3, 'feature_32'): 0.743710,
    (3, 'feature_33'): -0.221633,
    (3, 'feature_34'): 1.186568,
    (3, 'feature_35'): 0.784686,
    (3, 'feature_36'): -0.519836,
    (3, 'feature_37'): -0.142562,
    (3, 'feature_38'): -0.325055,
    (3, 'feature_39'): 0.200348,
    (3, 'feature_40'): -0.037105,
    (3, 'feature_41'): -0.360108,
    (3, 'feature_42'): 0.014951,
    (3, 'feature_43'): 0.522796,
    (3, 'feature_44'): -0.708808,
    (3, 'feature_45'): -0.353685,
    (3, 'feature_46'): 0.963778,
    (3, 'feature_47'): -0.024709,
    (3, 'feature_48'): 0.455486,
    (3, 'feature_49'): 0.089826,
    (3, 'feature_50'): 0.264640,
    (3, 'feature_51'): -0.306854,
    (3, 'feature_52'): -0.693773,
    (3, 'feature_53'): 0.011207,
    (3, 'feature_54'): -0.164694,
    (3, 'feature_55'): 0.055402,
    (3, 'feature_56'): -0.372127,
    (3, 'feature_57'): 0.913252,
    (3, 'feature_58'): -0.099935,
    (3, 'feature_59'): 0.269112,
    (3, 'feature_60'): 0.032934,
    (3, 'feature_61'): 0.845812,
    (3, 'feature_62'): -0.433302,
    (3, 'feature_63'): -0.327944,
    (3, 'feature_64'): -0.444309,
    (3, 'feature_65'): -0.341388,
    (3, 'feature_66'): 2.064613,
    (3, 'feature_67'): -0.165011,
    (3, 'feature_68'): -0.186260,
    (3, 'feature_69'): -0.067123,
    (3, 'feature_70'): -0.030187,
    (3, 'feature_71'): 0.156928,
    (3, 'feature_72'): 0.107753,
    (3, 'feature_73'): -0.020905,
    (3, 'feature_74'): 0.000754,
    (3, 'feature_75'): -0.018498,
    (3, 'feature_76'): 0.004122,
    (3, 'feature_77'): -0.014020,
    (3, 'feature_78'): 0.010026,
    (4, 'feature_00'): 0.846535,
    (4, 'feature_01'): 1.130689,
    (4, 'feature_02'): 0.780465,
    (4, 'feature_03'): -0.083662,
    (4, 'feature_04'): -0.267139,
    (4, 'feature_05'): -0.493514,
    (4, 'feature_06'): 0.243623,
    (4, 'feature_07'): 0.144075,
    (4, 'feature_08'): 1.445065,
    (4, 'feature_09'): 15.000000,
    (4, 'feature_10'): 1.000000,
    (4, 'feature_11'): 9.000000,
    (4, 'feature_12'): 0.396676,
    (4, 'feature_13'): 0.145045,
    (4, 'feature_14'): 0.192117,
    (4, 'feature_15'): -0.811509,
    (4, 'feature_16'): -0.282701,
    (4, 'feature_17'): -0.760028,
    (4, 'feature_18'): 2.730461,
    (4, 'feature_19'): 1.799058,
    (4, 'feature_20'): -0.270847,
    (4, 'feature_21'): 0.091326,
    (4, 'feature_22'): 0.553756,
    (4, 'feature_23'): -0.083852,
    (4, 'feature_24'): 1.894140,
    (4, 'feature_25'): 1.053614,
    (4, 'feature_26'): -1.628038,
    (4, 'feature_27'): -0.678881,
    (4, 'feature_28'): -0.167135,
    (4, 'feature_29'): -0.827343,
    (4, 'feature_30'): -0.904236,
    (4, 'feature_31'): 0.144417,
    (4, 'feature_32'): 1.986471,
    (4, 'feature_33'): -1.003394,
    (4, 'feature_34'): 1.557428,
    (4, 'feature_35'): 2.090332,
    (4, 'feature_36'): -0.864434,
    (4, 'feature_37'): 0.694344,
    (4, 'feature_38'): 0.184404,
    (4, 'feature_39'): -0.539752,
    (4, 'feature_40'): 0.592733,
    (4, 'feature_41'): -0.521836,
    (4, 'feature_42'): -0.398293,
    (4, 'feature_43'): -0.862385,
    (4, 'feature_44'): -1.345251,
    (4, 'feature_45'): -0.922414,
    (4, 'feature_46'): -1.235788,
    (4, 'feature_47'): -0.605658,
    (4, 'feature_48'): -0.105613,
    (4, 'feature_49'): -0.453961,
    (4, 'feature_50'): -0.794091,
    (4, 'feature_51'): 0.553430,
    (4, 'feature_52'): 0.095307,
    (4, 'feature_53'): -0.042550,
    (4, 'feature_54'): -0.796149,
    (4, 'feature_55'): -1.109538,
    (4, 'feature_56'): -0.745868,
    (4, 'feature_57'): -0.546390,
    (4, 'feature_58'): -0.909696,
    (4, 'feature_59'): -0.859667,
    (4, 'feature_60'): -0.608382,
    (4, 'feature_61'): 0.158662,
    (4, 'feature_62'): -0.441079,
    (4, 'feature_63'): -0.242470,
    (4, 'feature_64'): -0.417671,
    (4, 'feature_65'): 2.273387,
    (4, 'feature_66'): 1.778036,
    (4, 'feature_67'): 0.672469,
    (4, 'feature_68'): 0.360775,
    (4, 'feature_69'): 0.349170,
    (4, 'feature_70'): 0.005831,
    (4, 'feature_71'): -0.082374,
    (4, 'feature_72'): -0.101199,
    (4, 'feature_73'): 0.472107,
    (4, 'feature_74'): 0.452164,
    (4, 'feature_75'): 4.744576,
    (4, 'feature_76'): 3.507875,
    (4, 'feature_77'): 0.886147,
    (4, 'feature_78'): 0.578816,
    (5, 'feature_00'): 0.222053,
    (5, 'feature_01'): -0.731008,
    (5, 'feature_02'): 0.596135,
    (5, 'feature_03'): 0.370215,
    (5, 'feature_04'): -0.601732,
    (5, 'feature_05'): -1.449251,
    (5, 'feature_06'): -0.248804,
    (5, 'feature_07'): -1.796022,
    (5, 'feature_08'): -0.185894,
    (5, 'feature_09'): 2.000000,
    (5, 'feature_10'): 10.000000,
    (5, 'feature_11'): 171.000000,
    (5, 'feature_12'): 0.526284,
    (5, 'feature_13'): 0.531400,
    (5, 'feature_14'): 0.574471,
    (5, 'feature_15'): -0.931978,
    (5, 'feature_16'): -0.679761,
    (5, 'feature_17'): -0.603576,
    (5, 'feature_18'): -2.701849,
    (5, 'feature_19'): 3.294247,
    (5, 'feature_20'): 0.036411,
    (5, 'feature_21'): -0.109604,
    (5, 'feature_22'): 1.530177,
    (5, 'feature_23'): 0.056278,
    (5, 'feature_24'): 0.574376,
    (5, 'feature_25'): -0.002201,
    (5, 'feature_26'): -1.178187,
    (5, 'feature_27'): -0.033433,
    (5, 'feature_28'): 0.751161,
    (5, 'feature_29'): -0.794541,
    (5, 'feature_30'): -0.779101,
    (5, 'feature_31'): -0.095797,
    (5, 'feature_32'): 1.390482,
    (5, 'feature_33'): -0.149220,
    (5, 'feature_34'): 1.223787,
    (5, 'feature_35'): 1.317445,
    (5, 'feature_36'): 0.079455,
    (5, 'feature_37'): -2.514631,
    (5, 'feature_38'): -1.790180,
    (5, 'feature_39'): 2.162404,
    (5, 'feature_40'): 1.077106,
    (5, 'feature_41'): 0.980477,
    (5, 'feature_42'): 0.148064,
    (5, 'feature_43'): 0.378627,
    (5, 'feature_44'): -0.447215,
    (5, 'feature_45'): 2.237874,
    (5, 'feature_46'): 0.599394,
    (5, 'feature_47'): 0.957529,
    (5, 'feature_48'): 1.055233,
    (5, 'feature_49'): 0.818803,
    (5, 'feature_50'): 2.054656,
    (5, 'feature_51'): 0.890500,
    (5, 'feature_52'): 0.847599,
    (5, 'feature_53'): 0.348130,
    (5, 'feature_54'): -0.250847,
    (5, 'feature_55'): -0.786742,
    (5, 'feature_56'): 1.391605,
    (5, 'feature_57'): 0.618292,
    (5, 'feature_58'): 1.168340,
    (5, 'feature_59'): 0.574476,
    (5, 'feature_60'): 0.818746,
    (5, 'feature_61'): 0.845812,
    (5, 'feature_62'): -0.476493,
    (5, 'feature_63'): -0.303343,
    (5, 'feature_64'): -0.454548,
    (5, 'feature_65'): -1.633339,
    (5, 'feature_66'): 3.025509,
    (5, 'feature_67'): 0.383306,
    (5, 'feature_68'): 0.458776,
    (5, 'feature_69'): 0.645752,
    (5, 'feature_70'): 0.490595,
    (5, 'feature_71'): 0.603927,
    (5, 'feature_72'): 0.510773,
    (5, 'feature_73'): 0.019329,
    (5, 'feature_74'): 0.273267,
    (5, 'feature_75'): 0.431552,
    (5, 'feature_76'): 0.532204,
    (5, 'feature_77'): 0.047846,
    (5, 'feature_78'): 0.263817,
    (6, 'feature_00'): 0.389211,
    (6, 'feature_01'): -1.466745,
    (6, 'feature_02'): 0.770387,
    (6, 'feature_03'): 0.699883,
    (6, 'feature_04'): -0.201975,
    (6, 'feature_05'): -1.194083,
    (6, 'feature_06'): -0.212391,
    (6, 'feature_07'): -1.677560,
    (6, 'feature_08'): -0.122232,
    (6, 'feature_09'): 12.000000,
    (6, 'feature_10'): 4.000000,
    (6, 'feature_11'): 34.000000,
    (6, 'feature_12'): 1.367797,
    (6, 'feature_13'): 0.555664,
    (6, 'feature_14'): 2.071207,
    (6, 'feature_15'): -0.449704,
    (6, 'feature_16'): -0.250465,
    (6, 'feature_17'): -0.380029,
    (6, 'feature_18'): 0.601401,
    (6, 'feature_19'): 0.811935,
    (6, 'feature_20'): -0.400916,
    (6, 'feature_21'): -0.167209,
    (6, 'feature_22'): -0.653509,
    (6, 'feature_23'): -0.471450,
    (6, 'feature_24'): 1.049147,
    (6, 'feature_25'): 0.771161,
    (6, 'feature_26'): -0.513245,
    (6, 'feature_27'): -0.652036,
    (6, 'feature_28'): -0.667670,
    (6, 'feature_29'): -0.576041,
    (6, 'feature_30'): -0.667934,
    (6, 'feature_31'): -0.250472,
    (6, 'feature_32'): 0.181165,
    (6, 'feature_33'): 0.634343,
    (6, 'feature_34'): 0.466858,
    (6, 'feature_35'): 0.422863,
    (6, 'feature_36'): 2.094843,
    (6, 'feature_37'): 0.742300,
    (6, 'feature_38'): 0.188092,
    (6, 'feature_39'): 0.451048,
    (6, 'feature_40'): -0.557179,
    (6, 'feature_41'): 0.593798,
    (6, 'feature_42'): -3.055225,
    (6, 'feature_43'): -0.613451,
    (6, 'feature_44'): -0.368040,
    (6, 'feature_45'): -0.134118,
    (6, 'feature_46'): -1.405387,
    (6, 'feature_47'): -1.897366,
    (6, 'feature_48'): -0.014899,
    (6, 'feature_49'): -3.190421,
    (6, 'feature_50'): -0.113350,
    (6, 'feature_51'): -0.392338,
    (6, 'feature_52'): 0.611148,
    (6, 'feature_53'): -2.505984,
    (6, 'feature_54'): -1.529785,
    (6, 'feature_55'): -0.799840,
    (6, 'feature_56'): 0.559978,
    (6, 'feature_57'): -1.532759,
    (6, 'feature_58'): -3.779316,
    (6, 'feature_59'): -0.243394,
    (6, 'feature_60'): -3.061490,
    (6, 'feature_61'): 0.845812,
    (6, 'feature_62'): -0.276720,
    (6, 'feature_63'): -0.239221,
    (6, 'feature_64'): -0.365823,
    (6, 'feature_65'): 0.336105,
    (6, 'feature_66'): 1.114240,
    (6, 'feature_67'): 1.579894,
    (6, 'feature_68'): 0.426765,
    (6, 'feature_69'): 2.137358,
    (6, 'feature_70'): 1.825888,
    (6, 'feature_71'): 0.560038,
    (6, 'feature_72'): 2.180662,
    (6, 'feature_73'): 0.860839,
    (6, 'feature_74'): 1.006449,
    (6, 'feature_75'): 2.632397,
    (6, 'feature_76'): 2.924413,
    (6, 'feature_77'): 1.830547,
    (6, 'feature_78'): 2.694818,
    (7, 'feature_00'): 0.331582,
    (7, 'feature_01'): -0.980864,
    (7, 'feature_02'): -0.237925,
    (7, 'feature_03'): 0.520752,
    (7, 'feature_04'): -0.588696,
    (7, 'feature_05'): -1.643387,
    (7, 'feature_06'): -0.323883,
    (7, 'feature_07'): -1.721367,
    (7, 'feature_08'): -0.073730,
    (7, 'feature_09'): 11.000000,
    (7, 'feature_10'): 7.000000,
    (7, 'feature_11'): 76.000000,
    (7, 'feature_12'): 0.841892,
    (7, 'feature_13'): 1.373557,
    (7, 'feature_14'): 1.633267,
    (7, 'feature_15'): -0.657308,
    (7, 'feature_16'): -0.688009,
    (7, 'feature_17'): -0.607435,
    (7, 'feature_18'): 3.013578,
    (7, 'feature_19'): 0.810116,
    (7, 'feature_20'): 0.438517,
    (7, 'feature_21'): 0.024809,
    (7, 'feature_22'): 2.025378,
    (7, 'feature_23'): 0.961976,
    (7, 'feature_24'): 0.396860,
    (7, 'feature_25'): -0.512367,
    (7, 'feature_26'): -0.180533,
    (7, 'feature_27'): 0.457406,
    (7, 'feature_28'): 0.212277,
    (7, 'feature_29'): -0.740141,
    (7, 'feature_30'): -0.832856,
    (7, 'feature_31'): 0.031104,
    (7, 'feature_32'): 0.616571,
    (7, 'feature_33'): -0.120571,
    (7, 'feature_34'): 1.018815,
    (7, 'feature_35'): 1.142441,
    (7, 'feature_36'): 0.754844,
    (7, 'feature_37'): 0.534645,
    (7, 'feature_38'): 0.550978,
    (7, 'feature_39'): -0.542918,
    (7, 'feature_40'): 0.616660,
    (7, 'feature_41'): 0.416419,
    (7, 'feature_42'): -2.004647,
    (7, 'feature_43'): -0.328081,
    (7, 'feature_44'): -1.536123,
    (7, 'feature_45'): -0.662986,
    (7, 'feature_46'): -1.645698,
    (7, 'feature_47'): -2.486403,
    (7, 'feature_48'): -0.236390,
    (7, 'feature_49'): -3.216464,
    (7, 'feature_50'): -0.066224,
    (7, 'feature_51'): 1.617356,
    (7, 'feature_52'): 1.375026,
    (7, 'feature_53'): -2.356932,
    (7, 'feature_54'): -0.069708,
    (7, 'feature_55'): -1.957572,
    (7, 'feature_56'): -1.227010,
    (7, 'feature_57'): -1.349270,
    (7, 'feature_58'): -3.669671,
    (7, 'feature_59'): -0.183425,
    (7, 'feature_60'): -3.538151,
    (7, 'feature_61'): 0.845812,
    (7, 'feature_62'): -0.508019,
    (7, 'feature_63'): -0.320457,
    (7, 'feature_64'): -0.406539,
    (7, 'feature_65'): 3.896820,
    (7, 'feature_66'): 0.205845,
    (7, 'feature_67'): 1.096390,
    (7, 'feature_68'): 0.630964,
    (7, 'feature_69'): 2.326100,
    (7, 'feature_70'): 0.456069,
    (7, 'feature_71'): 1.188823,
    (7, 'feature_72'): 1.077103,
    (7, 'feature_73'): 0.814033,
    (7, 'feature_74'): 1.061581,
    (7, 'feature_75'): 1.759319,
    (7, 'feature_76'): 1.617359,
    (7, 'feature_77'): 1.198470,
    (7, 'feature_78'): 0.811223,
    (8, 'feature_00'): 0.405084,
    (8, 'feature_01'): -1.267321,
    (8, 'feature_02'): 0.366282,
    (8, 'feature_03'): 0.297634,
    (8, 'feature_04'): -0.752054,
    (8, 'feature_05'): -1.340931,
    (8, 'feature_06'): -0.171569,
    (8, 'feature_07'): -1.187821,
    (8, 'feature_08'): -0.155693,
    (8, 'feature_09'): 15.000000,
    (8, 'feature_10'): 1.000000,
    (8, 'feature_11'): 62.000000,
    (8, 'feature_12'): 0.087051,
    (8, 'feature_13'): 0.225471,
    (8, 'feature_14'): 0.532791,
    (8, 'feature_15'): -0.475189,
    (8, 'feature_16'): -0.835739,
    (8, 'feature_17'): -0.854600,
    (8, 'feature_18'): -0.131734,
    (8, 'feature_19'): 2.765579,
    (8, 'feature_20'): 1.072228,
    (8, 'feature_21'): 0.023580,
    (8, 'feature_22'): 0.429219,
    (8, 'feature_23'): 0.312847,
    (8, 'feature_24'): 1.659391,
    (8, 'feature_25'): 1.019123,
    (8, 'feature_26'): 0.485096,
    (8, 'feature_27'): -0.599773,
    (8, 'feature_28'): -0.893510,
    (8, 'feature_29'): -0.719832,
    (8, 'feature_30'): -0.722352,
    (8, 'feature_31'): 0.030333,
    (8, 'feature_32'): 0.611019,
    (8, 'feature_33'): 0.989214,
    (8, 'feature_34'): 1.238997,
    (8, 'feature_35'): 0.752547,
    (8, 'feature_36'): -2.112409,
    (8, 'feature_37'): -0.595686,
    (8, 'feature_38'): -0.636001,
    (8, 'feature_39'): -0.559532,
    (8, 'feature_40'): 0.533765,
    (8, 'feature_41'): -0.692805,
    (8, 'feature_42'): -1.557340,
    (8, 'feature_43'): -0.271428,
    (8, 'feature_44'): -1.078196,
    (8, 'feature_45'): 1.233133,
    (8, 'feature_46'): -0.155705,
    (8, 'feature_47'): -0.757759,
    (8, 'feature_48'): -0.595266,
    (8, 'feature_49'): -1.117017,
    (8, 'feature_50'): -0.507618,
    (8, 'feature_51'): -0.677647,
    (8, 'feature_52'): -0.180110,
    (8, 'feature_53'): -1.093041,
    (8, 'feature_54'): -0.276183,
    (8, 'feature_55'): 0.268834,
    (8, 'feature_56'): 0.634547,
    (8, 'feature_57'): -0.164922,
    (8, 'feature_58'): -0.669614,
    (8, 'feature_59'): -0.345253,
    (8, 'feature_60'): -0.569031,
    (8, 'feature_61'): 0.845812,
    (8, 'feature_62'): -0.484716,
    (8, 'feature_63'): -0.347901,
    (8, 'feature_64'): -0.333264,
    (8, 'feature_65'): -0.039495,
    (8, 'feature_66'): 2.215118,
    (8, 'feature_67'): 0.021817,
    (8, 'feature_68'): 0.228495,
    (8, 'feature_69'): 0.266363,
    (8, 'feature_70'): 0.264863,
    (8, 'feature_71'): 0.103209,
    (8, 'feature_72'): 0.616378,
    (8, 'feature_73'): -1.339614,
    (8, 'feature_74'): -1.300550,
    (8, 'feature_75'): -0.746375,
    (8, 'feature_76'): -0.837331,
    (8, 'feature_77'): -1.119476,
    (8, 'feature_78'): -1.035760,
    (9, 'feature_00'): 0.762038,
    (9, 'feature_01'): -1.548730,
    (9, 'feature_02'): 0.876251,
    (9, 'feature_03'): 0.632528,
    (9, 'feature_04'): -0.977852,
    (9, 'feature_05'): -1.495931,
    (9, 'feature_06'): -0.260931,
    (9, 'feature_07'): -1.745089,
    (9, 'feature_08'): -0.170608,
    (9, 'feature_09'): 11.000000,
    (9, 'feature_10'): 7.000000,
    (9, 'feature_11'): 76.000000,
    (9, 'feature_12'): -0.040100,
    (9, 'feature_13'): 1.551315,
    (9, 'feature_14'): 0.392774,
    (9, 'feature_15'): -0.556837,
    (9, 'feature_16'): -0.242261,
    (9, 'feature_17'): -0.299965,
    (9, 'feature_18'): -0.524776,
    (9, 'feature_19'): 2.630658,
    (9, 'feature_20'): -0.311980,
    (9, 'feature_21'): -0.371075,
    (9, 'feature_22'): -0.316324,
    (9, 'feature_23'): 0.014095,
    (9, 'feature_24'): 0.238056,
    (9, 'feature_25'): -0.659673,
    (9, 'feature_26'): 4.510432,
    (9, 'feature_27'): 0.961515,
    (9, 'feature_28'): -0.563246,
    (9, 'feature_29'): -0.682235,
    (9, 'feature_30'): -0.702045,
    (9, 'feature_31'): -0.245975,
    (9, 'feature_32'): 1.158022,
    (9, 'feature_33'): -0.128325,
    (9, 'feature_34'): 0.726704,
    (9, 'feature_35'): 1.226880,
    (9, 'feature_36'): -0.883750,
    (9, 'feature_37'): -0.405443,
    (9, 'feature_38'): -0.362939,
    (9, 'feature_39'): 0.002388,
    (9, 'feature_40'): 0.045426,
    (9, 'feature_41'): 0.926225,
    (9, 'feature_42'): -0.758307,
    (9, 'feature_43'): 0.327470,
    (9, 'feature_44'): 0.388994,
    (9, 'feature_45'): 0.406418,
    (9, 'feature_46'): 0.157923,
    (9, 'feature_47'): -0.218234,
    (9, 'feature_48'): 0.734677,
    (9, 'feature_49'): 0.126846,
    (9, 'feature_50'): 0.302810,
    (9, 'feature_51'): -0.014949,
    (9, 'feature_52'): 0.858962,
    (9, 'feature_53'): -0.269806,
    (9, 'feature_54'): -0.393128,
    (9, 'feature_55'): 0.113273,
    (9, 'feature_56'): 0.421524,
    (9, 'feature_57'): 0.650739,
    (9, 'feature_58'): -0.408629,
    (9, 'feature_59'): 1.330231,
    (9, 'feature_60'): 0.117725,
    (9, 'feature_61'): 0.845812,
    (9, 'feature_62'): -0.400512,
    (9, 'feature_63'): -0.215387,
    (9, 'feature_64'): -0.313898,
    (9, 'feature_65'): 0.083414,
    (9, 'feature_66'): 1.641663,
    (9, 'feature_67'): -0.040918,
    (9, 'feature_68'): 0.312329,
    (9, 'feature_69'): 0.326295,
    (9, 'feature_70'): -0.020937,
    (9, 'feature_71'): 1.077924,
    (9, 'feature_72'): 0.510471,
    (9, 'feature_73'): -0.086699,
    (9, 'feature_74'): -0.018418,
    (9, 'feature_75'): 0.875791,
    (9, 'feature_76'): 0.933501,
    (9, 'feature_77'): 0.117917,
    (9, 'feature_78'): 0.196025,
    (10, 'feature_00'): 0.131412,
    (10, 'feature_01'): -1.397453,
    (10, 'feature_02'): 0.773413,
    (10, 'feature_03'): 0.395383,
    (10, 'feature_04'): -0.859588,
    (10, 'feature_05'): -1.778774,
    (10, 'feature_06'): -0.292484,
    (10, 'feature_07'): -1.611303,
    (10, 'feature_08'): -0.168501,
    (10, 'feature_09'): 42.000000,
    (10, 'feature_10'): 5.000000,
    (10, 'feature_11'): 150.000000,
    (10, 'feature_12'): 0.381749,
    (10, 'feature_13'): 0.649498,
    (10, 'feature_14'): 0.662976,
    (10, 'feature_15'): -0.755882,
    (10, 'feature_16'): -0.463312,
    (10, 'feature_17'): -0.679050,
    (10, 'feature_18'): 3.142383,
    (10, 'feature_19'): 0.360051,
    (10, 'feature_20'): 0.431979,
    (10, 'feature_21'): -0.154984,
    (10, 'feature_22'): 0.565461,
    (10, 'feature_23'): 0.838108,
    (10, 'feature_24'): 0.147757,
    (10, 'feature_25'): -1.018841,
    (10, 'feature_26'): 1.104064,
    (10, 'feature_27'): 0.662018,
    (10, 'feature_28'): -0.240863,
    (10, 'feature_29'): -0.739297,
    (10, 'feature_30'): -0.610508,
    (10, 'feature_31'): -0.102603,
    (10, 'feature_32'): 1.482801,
    (10, 'feature_33'): 0.040455,
    (10, 'feature_34'): 1.119197,
    (10, 'feature_35'): 1.047508,
    (10, 'feature_36'): -0.151496,
    (10, 'feature_37'): 0.924821,
    (10, 'feature_38'): 0.760554,
    (10, 'feature_39'): 1.069762,
    (10, 'feature_40'): 0.726992,
    (10, 'feature_41'): 0.877944,
    (10, 'feature_42'): -0.862376,
    (10, 'feature_43'): -0.880584,
    (10, 'feature_44'): -0.048455,
    (10, 'feature_45'): 0.032249,
    (10, 'feature_46'): -0.871028,
    (10, 'feature_47'): -0.521943,
    (10, 'feature_48'): 0.270323,
    (10, 'feature_49'): 0.014690,
    (10, 'feature_50'): 0.561273,
    (10, 'feature_51'): 0.610507,
    (10, 'feature_52'): -0.363653,
    (10, 'feature_53'): -0.316054,
    (10, 'feature_54'): 0.686582,
    (10, 'feature_55'): -0.513060,
    (10, 'feature_56'): -0.537574,
    (10, 'feature_57'): -1.176764,
    (10, 'feature_58'): -0.943699,
    (10, 'feature_59'): 0.055196,
    (10, 'feature_60'): -0.160633,
    (10, 'feature_61'): 0.845812,
    (10, 'feature_62'): -0.441816,
    (10, 'feature_63'): -0.232803,
    (10, 'feature_64'): -0.506227,
    (10, 'feature_65'): 2.886308,
    (10, 'feature_66'): 0.909459,
    (10, 'feature_67'): 0.428922,
    (10, 'feature_68'): 0.231670,
    (10, 'feature_69'): 0.295794,
    (10, 'feature_70'): 0.289000,
    (10, 'feature_71'): 0.788486,
    (10, 'feature_72'): 0.533359,
    (10, 'feature_73'): -0.238842,
    (10, 'feature_74'): -0.198448,
    (10, 'feature_75'): 0.029304,
    (10, 'feature_76'): -0.014821,
    (10, 'feature_77'): -0.031596,
    (10, 'feature_78'): -0.081950,
    (11, 'feature_00'): 0.057355,
    (11, 'feature_01'): -1.289969,
    (11, 'feature_02'): 0.887498,
    (11, 'feature_03'): 0.795222,
    (11, 'feature_04'): -0.738843,
    (11, 'feature_05'): -3.005524,
    (11, 'feature_06'): -0.397802,
    (11, 'feature_07'): -1.979634,
    (11, 'feature_08'): -0.213251,
    (11, 'feature_09'): 46.000000,
    (11, 'feature_10'): 3.000000,
    (11, 'feature_11'): 261.000000,
    (11, 'feature_12'): 0.149495,
    (11, 'feature_13'): 0.505217,
    (11, 'feature_14'): 0.493464,
    (11, 'feature_15'): -0.648478,
    (11, 'feature_16'): -0.557657,
    (11, 'feature_17'): -0.787367,
    (11, 'feature_18'): 0.154225,
    (11, 'feature_19'): 1.935821,
    (11, 'feature_20'): -0.275741,
    (11, 'feature_21'): -0.199669,
    (11, 'feature_22'): 0.402847,
    (11, 'feature_23'): 0.203268,
    (11, 'feature_24'): -0.636067,
    (11, 'feature_25'): -1.327253,
    (11, 'feature_26'): 0.022226,
    (11, 'feature_27'): 0.016679,
    (11, 'feature_28'): -0.117278,
    (11, 'feature_29'): -0.941812,
    (11, 'feature_30'): -1.103308,
    (11, 'feature_31'): -0.087013,
    (11, 'feature_32'): 1.102968,
    (11, 'feature_33'): -0.027720,
    (11, 'feature_34'): 1.561425,
    (11, 'feature_35'): 1.136958,
    (11, 'feature_36'): -0.804259,
    (11, 'feature_37'): -0.703362,
    (11, 'feature_38'): -0.598695,
    (11, 'feature_39'): 0.077247,
    (11, 'feature_40'): 0.270263,
    (11, 'feature_41'): 0.111759,
    (11, 'feature_42'): -1.049578,
    (11, 'feature_43'): 0.256383,
    (11, 'feature_44'): -0.242472,
    (11, 'feature_45'): 0.513169,
    (11, 'feature_46'): -0.211163,
    (11, 'feature_47'): -0.540751,
    (11, 'feature_48'): 0.143308,
    (11, 'feature_49'): -0.522227,
    (11, 'feature_50'): 0.423062,
    (11, 'feature_51'): 0.423409,
    (11, 'feature_52'): 0.333865,
    (11, 'feature_53'): -0.353631,
    (11, 'feature_54'): 0.292213,
    (11, 'feature_55'): -0.457139,
    (11, 'feature_56'): 0.293237,
    (11, 'feature_57'): -0.184194,
    (11, 'feature_58'): -0.673231,
    (11, 'feature_59'): 0.501729,
    (11, 'feature_60'): -0.370953,
    (11, 'feature_61'): 0.845812,
    (11, 'feature_62'): -0.492695,
    (11, 'feature_63'): -0.246460,
    (11, 'feature_64'): -0.437182,
    (11, 'feature_65'): -0.353016,
    (11, 'feature_66'): 1.867050,
    (11, 'feature_67'): 0.297958,
    (11, 'feature_68'): 0.104314,
    (11, 'feature_69'): 0.503973,
    (11, 'feature_70'): -0.068805,
    (11, 'feature_71'): 0.636612,
    (11, 'feature_72'): 0.304170,
    (11, 'feature_73'): -0.044205,
    (11, 'feature_74'): -0.060511,
    (11, 'feature_75'): 0.812487,
    (11, 'feature_76'): 0.733677,
    (11, 'feature_77'): 0.105930,
    (11, 'feature_78'): 0.131311,
    (12, 'feature_00'): 0.447192,
    (12, 'feature_01'): -0.974968,
    (12, 'feature_02'): 0.469129,
    (12, 'feature_03'): 0.381191,
    (12, 'feature_04'): -0.597162,
    (12, 'feature_05'): -1.221360,
    (12, 'feature_06'): -0.287159,
    (12, 'feature_07'): -0.889688,
    (12, 'feature_08'): -0.179172,
    (12, 'feature_09'): 70.000000,
    (12, 'feature_10'): 7.000000,
    (12, 'feature_11'): 410.000000,
    (12, 'feature_12'): -0.140310,
    (12, 'feature_13'): 0.029852,
    (12, 'feature_14'): -0.035182,
    (12, 'feature_15'): -0.693815,
    (12, 'feature_16'): -0.399995,
    (12, 'feature_17'): -0.496888,
    (12, 'feature_18'): -0.010693,
    (12, 'feature_19'): 3.120752,
    (12, 'feature_20'): -0.366607,
    (12, 'feature_21'): 0.403380,
    (12, 'feature_22'): 1.102059,
    (12, 'feature_23'): 2.134708,
    (12, 'feature_24'): 1.100124,
    (12, 'feature_25'): 0.658152,
    (12, 'feature_26'): 0.516078,
    (12, 'feature_27'): -0.795487,
    (12, 'feature_28'): -1.097678,
    (12, 'feature_29'): -0.161163,
    (12, 'feature_30'): -0.325182,
    (12, 'feature_31'): 0.453783,
    (12, 'feature_32'): 0.948065,
    (12, 'feature_33'): -0.592020,
    (12, 'feature_34'): 0.846035,
    (12, 'feature_35'): 1.171356,
    (12, 'feature_36'): -0.786623,
    (12, 'feature_37'): 0.283496,
    (12, 'feature_38'): 0.030636,
    (12, 'feature_39'): -0.090911,
    (12, 'feature_40'): 0.009334,
    (12, 'feature_41'): -0.303236,
    (12, 'feature_42'): -1.062154,
    (12, 'feature_43'): -0.963427,
    (12, 'feature_44'): -0.916100,
    (12, 'feature_45'): -0.086152,
    (12, 'feature_46'): 0.337232,
    (12, 'feature_47'): -0.471436,
    (12, 'feature_48'): -0.623850,
    (12, 'feature_49'): -0.778734,
    (12, 'feature_50'): -1.362748,
    (12, 'feature_51'): 0.359186,
    (12, 'feature_52'): -0.803905,
    (12, 'feature_53'): -1.491101,
    (12, 'feature_54'): -1.019684,
    (12, 'feature_55'): -0.405803,
    (12, 'feature_56'): 0.765472,
    (12, 'feature_57'): 0.540448,
    (12, 'feature_58'): -0.947660,
    (12, 'feature_59'): -0.994098,
    (12, 'feature_60'): -1.015702,
    (12, 'feature_61'): 0.845812,
    (12, 'feature_62'): -0.360259,
    (12, 'feature_63'): -0.196681,
    (12, 'feature_64'): -0.232234,
    (12, 'feature_65'): -0.497316,
    (12, 'feature_66'): 2.651331,
    (12, 'feature_67'): -0.274755,
    (12, 'feature_68'): 0.161281,
    (12, 'feature_69'): -0.042450,
    (12, 'feature_70'): -0.155651,
    (12, 'feature_71'): -0.113095,
    (12, 'feature_72'): -0.010122,
    (12, 'feature_73'): -0.328905,
    (12, 'feature_74'): -0.312075,
    (12, 'feature_75'): -0.035740,
    (12, 'feature_76'): -0.063520,
    (12, 'feature_77'): -0.258419,
    (12, 'feature_78'): -0.221457,
    (13, 'feature_00'): -0.049996,
    (13, 'feature_01'): -1.646345,
    (13, 'feature_02'): 0.727465,
    (13, 'feature_03'): 0.711563,
    (13, 'feature_04'): -1.287891,
    (13, 'feature_05'): -1.087673,
    (13, 'feature_06'): -0.256744,
    (13, 'feature_07'): -1.293718,
    (13, 'feature_08'): -0.207574,
    (13, 'feature_09'): 50.000000,
    (13, 'feature_10'): 1.000000,
    (13, 'feature_11'): 522.000000,
    (13, 'feature_12'): 0.059823,
    (13, 'feature_13'): 0.280075,
    (13, 'feature_14'): 0.527669,
    (13, 'feature_15'): -0.814327,
    (13, 'feature_16'): -0.584732,
    (13, 'feature_17'): -0.625103,
    (13, 'feature_18'): -1.448395,
    (13, 'feature_19'): 2.285267,
    (13, 'feature_20'): 0.541000,
    (13, 'feature_21'): -0.042069,
    (13, 'feature_22'): -0.104742,
    (13, 'feature_23'): 0.471399,
    (13, 'feature_24'): 0.926332,
    (13, 'feature_25'): 0.654691,
    (13, 'feature_26'): 2.650028,
    (13, 'feature_27'): -0.944433,
    (13, 'feature_28'): -1.133224,
    (13, 'feature_29'): -0.493989,
    (13, 'feature_30'): -0.828904,
    (13, 'feature_31'): -0.021921,
    (13, 'feature_32'): 0.793143,
    (13, 'feature_33'): -0.014391,
    (13, 'feature_34'): 1.265225,
    (13, 'feature_35'): 1.951662,
    (13, 'feature_36'): -0.410120,
    (13, 'feature_37'): -0.583524,
    (13, 'feature_38'): -1.049229,
    (13, 'feature_39'): -0.615172,
    (13, 'feature_40'): 0.339139,
    (13, 'feature_41'): -0.283050,
    (13, 'feature_42'): -1.853038,
    (13, 'feature_43'): 0.107598,
    (13, 'feature_44'): -1.065871,
    (13, 'feature_45'): 1.101762,
    (13, 'feature_46'): -0.397078,
    (13, 'feature_47'): -0.827778,
    (13, 'feature_48'): -0.695895,
    (13, 'feature_49'): -1.533773,
    (13, 'feature_50'): -1.419168,
    (13, 'feature_51'): 0.702534,
    (13, 'feature_52'): -0.101247,
    (13, 'feature_53'): -1.419248,
    (13, 'feature_54'): 0.262144,
    (13, 'feature_55'): -2.133317,
    (13, 'feature_56'): 0.822515,
    (13, 'feature_57'): -0.102241,
    (13, 'feature_58'): -1.121348,
    (13, 'feature_59'): -0.384512,
    (13, 'feature_60'): -1.113992,
    (13, 'feature_61'): 0.845812,
    (13, 'feature_62'): -0.394125,
    (13, 'feature_63'): -0.291228,
    (13, 'feature_64'): -0.325172,
    (13, 'feature_65'): -1.381803,
    (13, 'feature_66'): 2.277293,
    (13, 'feature_67'): 0.221114,
    (13, 'feature_68'): 0.486451,
    (13, 'feature_69'): 0.729119,
    (13, 'feature_70'): -0.095353,
    (13, 'feature_71'): -0.043367,
    (13, 'feature_72'): 0.011520,
    (13, 'feature_73'): -0.281372,
    (13, 'feature_74'): -0.122481,
    (13, 'feature_75'): -0.143918,
    (13, 'feature_76'): -0.042708,
    (13, 'feature_77'): -0.118942,
    (13, 'feature_78'): -0.032214,
    (14, 'feature_00'): 0.834964,
    (14, 'feature_01'): -1.420285,
    (14, 'feature_02'): 0.270860,
    (14, 'feature_03'): 0.527587,
    (14, 'feature_04'): -1.207316,
    (14, 'feature_05'): -1.615713,
    (14, 'feature_06'): -0.350009,
    (14, 'feature_07'): -1.293973,
    (14, 'feature_08'): -0.238421,
    (14, 'feature_09'): 44.000000,
    (14, 'feature_10'): 3.000000,
    (14, 'feature_11'): 16.000000,
    (14, 'feature_12'): -0.062836,
    (14, 'feature_13'): 0.637509,
    (14, 'feature_14'): 0.565514,
    (14, 'feature_15'): -0.535134,
    (14, 'feature_16'): -0.403536,
    (14, 'feature_17'): -0.619083,
    (14, 'feature_18'): 0.135076,
    (14, 'feature_19'): 2.216341,
    (14, 'feature_20'): -0.177940,
    (14, 'feature_21'): -0.283173,
    (14, 'feature_22'): -0.829448,
    (14, 'feature_23'): -0.233654,
    (14, 'feature_24'): 0.380845,
    (14, 'feature_25'): -0.710810,
    (14, 'feature_26'): -0.451267,
    (14, 'feature_27'): -0.648384,
    (14, 'feature_28'): -0.564665,
    (14, 'feature_29'): -0.542070,
    (14, 'feature_30'): -0.559757,
    (14, 'feature_31'): -0.277081,
    (14, 'feature_32'): 0.581695,
    (14, 'feature_33'): -0.236101,
    (14, 'feature_34'): 1.427640,
    (14, 'feature_35'): 0.856006,
    (14, 'feature_36'): -0.300397,
    (14, 'feature_37'): 0.261973,
    (14, 'feature_38'): 0.110236,
    (14, 'feature_39'): -0.379387,
    (14, 'feature_40'): 0.285512,
    (14, 'feature_41'): 0.513804,
    (14, 'feature_42'): -1.102728,
    (14, 'feature_43'): -0.880483,
    (14, 'feature_44'): -0.876181,
    (14, 'feature_45'): -0.241279,
    (14, 'feature_46'): -0.500659,
    (14, 'feature_47'): -1.403745,
    (14, 'feature_48'): -0.465010,
    (14, 'feature_49'): -1.192685,
    (14, 'feature_50'): 0.187677,
    (14, 'feature_51'): -0.265245,
    (14, 'feature_52'): 1.172362,
    (14, 'feature_53'): -1.256422,
    (14, 'feature_54'): -0.169610,
    (14, 'feature_55'): -1.212051,
    (14, 'feature_56'): -0.555178,
    (14, 'feature_57'): -0.853663,
    (14, 'feature_58'): -1.660063,
    (14, 'feature_59'): -0.555834,
    (14, 'feature_60'): -1.263081,
    (14, 'feature_61'): 0.845812,
    (14, 'feature_62'): -0.298642,
    (14, 'feature_63'): -0.325511,
    (14, 'feature_64'): -0.415582,
    (14, 'feature_65'): 0.270925,
    (14, 'feature_66'): 1.689364,
    (14, 'feature_67'): 0.026339,
    (14, 'feature_68'): 0.717972,
    (14, 'feature_69'): 0.362983,
    (14, 'feature_70'): -0.161216,
    (14, 'feature_71'): 0.367991,
    (14, 'feature_72'): 0.111735,
    (14, 'feature_73'): -0.022414,
    (14, 'feature_74'): -0.047921,
    (14, 'feature_75'): 0.030158,
    (14, 'feature_76'): 0.042765,
    (14, 'feature_77'): 0.095919,
    (14, 'feature_78'): 0.086101,
    (15, 'feature_00'): 0.593546,
    (15, 'feature_01'): -1.627998,
    (15, 'feature_02'): 0.694613,
    (15, 'feature_03'): 0.884077,
    (15, 'feature_04'): -0.522733,
    (15, 'feature_05'): -1.218660,
    (15, 'feature_06'): -0.252406,
    (15, 'feature_07'): -0.952353,
    (15, 'feature_08'): -0.134163,
    (15, 'feature_09'): 9.000000,
    (15, 'feature_10'): 3.000000,
    (15, 'feature_11'): 25.000000,
    (15, 'feature_12'): 0.045092,
    (15, 'feature_13'): 0.294743,
    (15, 'feature_14'): 0.547810,
    (15, 'feature_15'): -0.424908,
    (15, 'feature_16'): -0.679272,
    (15, 'feature_17'): -1.267771,
    (15, 'feature_18'): 0.541079,
    (15, 'feature_19'): 3.743382,
    (15, 'feature_20'): -0.053083,
    (15, 'feature_21'): -0.235118,
    (15, 'feature_22'): -0.340143,
    (15, 'feature_23'): -0.433672,
    (15, 'feature_24'): 0.938443,
    (15, 'feature_25'): 0.341925,
    (15, 'feature_26'): -0.840059,
    (15, 'feature_27'): -0.611895,
    (15, 'feature_28'): -0.284747,
    (15, 'feature_29'): -0.685350,
    (15, 'feature_30'): -0.974478,
    (15, 'feature_31'): -0.170380,
    (15, 'feature_32'): 0.999059,
    (15, 'feature_33'): -0.456736,
    (15, 'feature_34'): 1.114686,
    (15, 'feature_35'): 1.212323,
    (15, 'feature_36'): -0.869710,
    (15, 'feature_37'): -1.134226,
    (15, 'feature_38'): -1.076435,
    (15, 'feature_39'): -0.025108,
    (15, 'feature_40'): 0.269595,
    (15, 'feature_41'): -0.023570,
    (15, 'feature_42'): 0.160629,
    (15, 'feature_43'): -0.086349,
    (15, 'feature_44'): 0.197314,
    (15, 'feature_45'): 1.228289,
    (15, 'feature_46'): 1.163921,
    (15, 'feature_47'): 0.092909,
    (15, 'feature_48'): 0.062865,
    (15, 'feature_49'): 0.080528,
    (15, 'feature_50'): -0.390398,
    (15, 'feature_51'): 0.094535,
    (15, 'feature_52'): -0.036114,
    (15, 'feature_53'): 0.333578,
    (15, 'feature_54'): 0.575897,
    (15, 'feature_55'): 0.122738,
    (15, 'feature_56'): 0.704206,
    (15, 'feature_57'): 1.292183,
    (15, 'feature_58'): 0.254237,
    (15, 'feature_59'): 0.422791,
    (15, 'feature_60'): 0.389525,
    (15, 'feature_61'): 0.845812,
    (15, 'feature_62'): -0.423270,
    (15, 'feature_63'): -0.298414,
    (15, 'feature_64'): -0.229703,
    (15, 'feature_65'): -0.082812,
    (15, 'feature_66'): 2.813147,
    (15, 'feature_67'): 0.025732,
    (15, 'feature_68'): 0.079876,
    (15, 'feature_69'): 0.205215,
    (15, 'feature_70'): 0.082280,
    (15, 'feature_71'): 0.446336,
    (15, 'feature_72'): 0.605172,
    (15, 'feature_73'): 0.181571,
    (15, 'feature_74'): 0.206644,
    (15, 'feature_75'): 0.014879,
    (15, 'feature_76'): 0.131528,
    (15, 'feature_77'): 0.134184,
    (15, 'feature_78'): 0.267626,
    (16, 'feature_00'): 0.513185,
    (16, 'feature_01'): -1.698943,
    (16, 'feature_02'): 0.493294,
    (16, 'feature_03'): 0.312506,
    (16, 'feature_04'): -0.383138,
    (16, 'feature_05'): -1.607518,
    (16, 'feature_06'): -0.317070,
    (16, 'feature_07'): -1.178574,
    (16, 'feature_08'): -0.169058,
    (16, 'feature_09'): 11.000000,
    (16, 'feature_10'): 7.000000,
    (16, 'feature_11'): 76.000000,
    (16, 'feature_12'): 0.312896,
    (16, 'feature_13'): 1.163149,
    (16, 'feature_14'): 1.144747,
    (16, 'feature_15'): -0.559317,
    (16, 'feature_16'): -0.543196,
    (16, 'feature_17'): -0.622756,
    (16, 'feature_18'): -0.342666,
    (16, 'feature_19'): 2.313435,
    (16, 'feature_20'): 1.256517,
    (16, 'feature_21'): 0.057047,
    (16, 'feature_22'): 2.599458,
    (16, 'feature_23'): 1.383833,
    (16, 'feature_24'): 0.738702,
    (16, 'feature_25'): 0.260628,
    (16, 'feature_26'): -0.458928,
    (16, 'feature_27'): 0.787476,
    (16, 'feature_28'): 0.898393,
    (16, 'feature_29'): -0.740442,
    (16, 'feature_30'): -0.739630,
    (16, 'feature_31'): 0.056160,
    (16, 'feature_32'): 0.647328,
    (16, 'feature_33'): -0.026853,
    (16, 'feature_34'): 1.402080,
    (16, 'feature_35'): 1.297595,
    (16, 'feature_36'): -0.352094,
    (16, 'feature_37'): 0.631133,
    (16, 'feature_38'): 0.399139,
    (16, 'feature_39'): -0.976668,
    (16, 'feature_40'): -0.267402,
    (16, 'feature_41'): -0.803724,
    (16, 'feature_42'): -2.011905,
    (16, 'feature_43'): -1.061397,
    (16, 'feature_44'): -2.231799,
    (16, 'feature_45'): -1.408002,
    (16, 'feature_46'): -1.177173,
    (16, 'feature_47'): -1.452285,
    (16, 'feature_48'): -2.287170,
    (16, 'feature_49'): -2.450551,
    (16, 'feature_50'): -2.266091,
    (16, 'feature_51'): -0.636608,
    (16, 'feature_52'): -1.908026,
    (16, 'feature_53'): -1.247796,
    (16, 'feature_54'): -0.747733,
    (16, 'feature_55'): -2.256730,
    (16, 'feature_56'): -2.007911,
    (16, 'feature_57'): -0.598694,
    (16, 'feature_58'): -1.553393,
    (16, 'feature_59'): -1.960696,
    (16, 'feature_60'): -3.146163,
    (16, 'feature_61'): 0.845812,
    (16, 'feature_62'): -0.301307,
    (16, 'feature_63'): -0.341728,
    (16, 'feature_64'): -0.421341,
    (16, 'feature_65'): 0.075327,
    (16, 'feature_66'): 1.365033,
    (16, 'feature_67'): 0.361131,
    (16, 'feature_68'): 1.334956,
    (16, 'feature_69'): 1.394068,
    (16, 'feature_70'): 0.098129,
    (16, 'feature_71'): 0.214567,
    (16, 'feature_72'): 0.293706,
    (16, 'feature_73'): -0.269273,
    (16, 'feature_74'): -0.399784,
    (16, 'feature_75'): 0.310643,
    (16, 'feature_76'): 0.258974,
    (16, 'feature_77'): -0.108604,
    (16, 'feature_78'): -0.144521,
    (17, 'feature_00'): 0.367899,
    (17, 'feature_01'): -1.261960,
    (17, 'feature_02'): 0.579211,
    (17, 'feature_03'): 0.818132,
    (17, 'feature_04'): -1.032748,
    (17, 'feature_05'): -1.521680,
    (17, 'feature_06'): -0.349059,
    (17, 'feature_07'): -1.822485,
    (17, 'feature_08'): -0.059879,
    (17, 'feature_09'): 64.000000,
    (17, 'feature_10'): 6.000000,
    (17, 'feature_11'): 376.000000,
    (17, 'feature_12'): 0.783015,
    (17, 'feature_13'): -0.024437,
    (17, 'feature_14'): 0.410799,
    (17, 'feature_15'): -0.708904,
    (17, 'feature_16'): -0.392838,
    (17, 'feature_17'): -0.510339,
    (17, 'feature_18'): 2.536790,
    (17, 'feature_19'): -0.564869,
    (17, 'feature_20'): 0.139689,
    (17, 'feature_21'): -0.118850,
    (17, 'feature_22'): 0.779616,
    (17, 'feature_23'): 1.849715,
    (17, 'feature_24'): 0.078094,
    (17, 'feature_25'): -0.819952,
    (17, 'feature_26'): 0.301938,
    (17, 'feature_27'): 0.970303,
    (17, 'feature_28'): 0.915253,
    (17, 'feature_29'): -0.721130,
    (17, 'feature_30'): -0.648520,
    (17, 'feature_31'): -0.256227,
    (17, 'feature_32'): 0.928317,
    (17, 'feature_33'): 0.051312,
    (17, 'feature_34'): 1.295360,
    (17, 'feature_35'): 1.493165,
    (17, 'feature_36'): 0.267041,
    (17, 'feature_37'): 0.774584,
    (17, 'feature_38'): 0.441447,
    (17, 'feature_39'): 1.324513,
    (17, 'feature_40'): 1.154610,
    (17, 'feature_41'): -0.076395,
    (17, 'feature_42'): 0.409973,
    (17, 'feature_43'): -0.386784,
    (17, 'feature_44'): -0.743005,
    (17, 'feature_45'): 0.077559,
    (17, 'feature_46'): -0.399545,
    (17, 'feature_47'): -0.008803,
    (17, 'feature_48'): 0.555134,
    (17, 'feature_49'): -0.013336,
    (17, 'feature_50'): 0.756014,
    (17, 'feature_51'): 0.818061,
    (17, 'feature_52'): 0.299871,
    (17, 'feature_53'): -0.051034,
    (17, 'feature_54'): 0.429055,
    (17, 'feature_55'): -0.846032,
    (17, 'feature_56'): -0.598798,
    (17, 'feature_57'): -0.847900,
    (17, 'feature_58'): -0.006441,
    (17, 'feature_59'): 0.587398,
    (17, 'feature_60'): 0.028402,
    (17, 'feature_61'): 0.845812,
    (17, 'feature_62'): -0.415217,
    (17, 'feature_63'): -0.222885,
    (17, 'feature_64'): -0.327436,
    (17, 'feature_65'): 1.834580,
    (17, 'feature_66'): -0.043218,
    (17, 'feature_67'): 0.948086,
    (17, 'feature_68'): -0.019859,
    (17, 'feature_69'): 0.343284,
    (17, 'feature_70'): 0.784009,
    (17, 'feature_71'): -0.011706,
    (17, 'feature_72'): 0.392321,
    (17, 'feature_73'): -0.143661,
    (17, 'feature_74'): -0.241732,
    (17, 'feature_75'): 0.015073,
    (17, 'feature_76'): -0.019865,
    (17, 'feature_77'): -0.172302,
    (17, 'feature_78'): -0.140224,
    (18, 'feature_00'): 0.069828,
    (18, 'feature_01'): -1.036350,
    (18, 'feature_02'): 0.716388,
    (18, 'feature_03'): 0.501004,
    (18, 'feature_04'): -1.201133,
    (18, 'feature_05'): -1.111539,
    (18, 'feature_06'): -0.214615,
    (18, 'feature_07'): -1.286781,
    (18, 'feature_08'): -0.120830,
    (18, 'feature_09'): 15.000000,
    (18, 'feature_10'): 1.000000,
    (18, 'feature_11'): 66.000000,
    (18, 'feature_12'): 0.110751,
    (18, 'feature_13'): 0.270946,
    (18, 'feature_14'): 0.337643,
    (18, 'feature_15'): -0.746952,
    (18, 'feature_16'): -0.788240,
    (18, 'feature_17'): -0.510983,
    (18, 'feature_18'): -1.192251,
    (18, 'feature_19'): 2.203202,
    (18, 'feature_20'): 0.236927,
    (18, 'feature_21'): -0.105472,
    (18, 'feature_22'): -0.229643,
    (18, 'feature_23'): -0.140480,
    (18, 'feature_24'): 1.612318,
    (18, 'feature_25'): 1.313467,
    (18, 'feature_26'): 0.193542,
    (18, 'feature_27'): -0.530984,
    (18, 'feature_28'): -1.197448,
    (18, 'feature_29'): -0.662578,
    (18, 'feature_30'): -0.837439,
    (18, 'feature_31'): -0.167568,
    (18, 'feature_32'): 0.961276,
    (18, 'feature_33'): -0.390217,
    (18, 'feature_34'): 1.242109,
    (18, 'feature_35'): 0.960752,
    (18, 'feature_36'): -0.086533,
    (18, 'feature_37'): -1.384997,
    (18, 'feature_38'): -1.305600,
    (18, 'feature_39'): 0.325380,
    (18, 'feature_40'): 0.438471,
    (18, 'feature_41'): 0.383584,
    (18, 'feature_42'): -1.050405,
    (18, 'feature_43'): 0.242715,
    (18, 'feature_44'): 0.146423,
    (18, 'feature_45'): 1.744651,
    (18, 'feature_46'): -0.064960,
    (18, 'feature_47'): -0.304949,
    (18, 'feature_48'): 0.281806,
    (18, 'feature_49'): 0.025375,
    (18, 'feature_50'): 1.090977,
    (18, 'feature_51'): -0.018463,
    (18, 'feature_52'): -0.696389,
    (18, 'feature_53'): -0.774039,
    (18, 'feature_54'): -0.207530,
    (18, 'feature_55'): -0.009946,
    (18, 'feature_56'): 1.597315,
    (18, 'feature_57'): 0.341964,
    (18, 'feature_58'): -0.313677,
    (18, 'feature_59'): 0.279215,
    (18, 'feature_60'): 0.057488,
    (18, 'feature_61'): 0.845812,
    (18, 'feature_62'): -0.275986,
    (18, 'feature_63'): -0.340407,
    (18, 'feature_64'): -0.409595,
    (18, 'feature_65'): -0.245011,
    (18, 'feature_66'): 2.160543,
    (18, 'feature_67'): 0.098808,
    (18, 'feature_68'): 0.018593,
    (18, 'feature_69'): 0.266249,
    (18, 'feature_70'): 0.114283,
    (18, 'feature_71'): 0.425213,
    (18, 'feature_72'): 0.400775,
    (18, 'feature_73'): -0.221758,
    (18, 'feature_74'): 0.040891,
    (18, 'feature_75'): -0.108874,
    (18, 'feature_76'): 0.083755,
    (18, 'feature_77'): -0.145112,
    (18, 'feature_78'): 0.081315,
    (19, 'feature_00'): 0.550264,
    (19, 'feature_01'): -1.361815,
    (19, 'feature_02'): -0.291164,
    (19, 'feature_03'): 0.899434,
    (19, 'feature_04'): -0.691335,
    (19, 'feature_05'): -1.648774,
    (19, 'feature_06'): -0.245715,
    (19, 'feature_07'): -1.205020,
    (19, 'feature_08'): -0.056508,
    (19, 'feature_09'): 4.000000,
    (19, 'feature_10'): 3.000000,
    (19, 'feature_11'): 11.000000,
    (19, 'feature_12'): -0.008306,
    (19, 'feature_13'): 0.108467,
    (19, 'feature_14'): 0.376775,
    (19, 'feature_15'): -0.583013,
    (19, 'feature_16'): -0.814852,
    (19, 'feature_17'): -0.993919,
    (19, 'feature_18'): -0.340607,
    (19, 'feature_19'): 2.216749,
    (19, 'feature_20'): 0.215951,
    (19, 'feature_21'): -0.094488,
    (19, 'feature_22'): 2.210197,
    (19, 'feature_23'): 2.121698,
    (19, 'feature_24'): 0.575223,
    (19, 'feature_25'): -0.628442,
    (19, 'feature_26'): 0.619969,
    (19, 'feature_27'): 1.606419,
    (19, 'feature_28'): 2.103366,
    (19, 'feature_29'): -0.692766,
    (19, 'feature_30'): -0.581948,
    (19, 'feature_31'): -0.066365,
    (19, 'feature_32'): 1.514219,
    (19, 'feature_33'): -0.179483,
    (19, 'feature_34'): 1.229467,
    (19, 'feature_35'): 0.662655,
    (19, 'feature_36'): 0.033775,
    (19, 'feature_37'): -0.646762,
    (19, 'feature_38'): -0.787084,
    (19, 'feature_39'): 0.748400,
    (19, 'feature_40'): 1.071225,
    (19, 'feature_41'): 0.765272,
    (19, 'feature_42'): 0.502499,
    (19, 'feature_43'): 0.366061,
    (19, 'feature_44'): 0.171653,
    (19, 'feature_45'): 0.434374,
    (19, 'feature_46'): 0.858138,
    (19, 'feature_47'): 0.048773,
    (19, 'feature_48'): 0.622587,
    (19, 'feature_49'): 0.548224,
    (19, 'feature_50'): 1.545884,
    (19, 'feature_51'): -0.034799,
    (19, 'feature_52'): 0.772744,
    (19, 'feature_53'): 0.797768,
    (19, 'feature_54'): 0.191690,
    (19, 'feature_55'): 0.346545,
    (19, 'feature_56'): 0.620054,
    (19, 'feature_57'): 0.804966,
    (19, 'feature_58'): 0.050425,
    (19, 'feature_59'): 1.037532,
    (19, 'feature_60'): 0.500157,
    (19, 'feature_61'): 0.845812,
    (19, 'feature_62'): -0.402260,
    (19, 'feature_63'): -0.371089,
    (19, 'feature_64'): -0.494892,
    (19, 'feature_65'): -0.343625,
    (19, 'feature_66'): 0.455223,
    (19, 'feature_67'): -0.028238,
    (19, 'feature_68'): -0.040929,
    (19, 'feature_69'): 0.307997,
    (19, 'feature_70'): 0.023054,
    (19, 'feature_71'): 0.244574,
    (19, 'feature_72'): 0.468660,
    (19, 'feature_73'): -2.208346,
    (19, 'feature_74'): -1.826072,
    (19, 'feature_75'): -1.052810,
    (19, 'feature_76'): -1.948575,
    (19, 'feature_77'): -1.869029,
    (19, 'feature_78'): -1.576337,
    (20, 'feature_00'): 0.711746,
    (20, 'feature_01'): -1.369403,
    (20, 'feature_02'): 0.642936,
    (20, 'feature_03'): 0.181611,
    (20, 'feature_04'): -0.648351,
    (20, 'feature_05'): -1.491248,
    (20, 'feature_06'): -0.153358,
    (20, 'feature_07'): -0.894359,
    (20, 'feature_08'): -0.131728,
    (20, 'feature_09'): 30.000000,
    (20, 'feature_10'): 10.000000,
    (20, 'feature_11'): 50.000000,
    (20, 'feature_12'): 0.835981,
    (20, 'feature_13'): 0.105016,
    (20, 'feature_14'): 0.543251,
    (20, 'feature_15'): -0.758151,
    (20, 'feature_16'): -0.648221,
    (20, 'feature_17'): -0.614564,
    (20, 'feature_18'): 0.336264,
    (20, 'feature_19'): 1.460190,
    (20, 'feature_20'): -0.603935,
    (20, 'feature_21'): -0.182305,
    (20, 'feature_22'): -1.202538,
    (20, 'feature_23'): -0.716593,
    (20, 'feature_24'): 2.765215,
    (20, 'feature_25'): 2.413357,
    (20, 'feature_26'): 1.175882,
    (20, 'feature_27'): -1.600121,
    (20, 'feature_28'): -2.158724,
    (20, 'feature_29'): -0.576701,
    (20, 'feature_30'): -1.080411,
    (20, 'feature_31'): -0.156780,
    (20, 'feature_32'): -1.718901,
    (20, 'feature_33'): 2.158590,
    (20, 'feature_34'): -1.691001,
    (20, 'feature_35'): -1.490224,
    (20, 'feature_36'): 1.297270,
    (20, 'feature_37'): -0.029845,
    (20, 'feature_38'): -0.157744,
    (20, 'feature_39'): -0.482684,
    (20, 'feature_40'): -1.184660,
    (20, 'feature_41'): 0.741444,
    (20, 'feature_42'): -1.387586,
    (20, 'feature_43'): 0.016148,
    (20, 'feature_44'): -0.223520,
    (20, 'feature_45'): 1.823546,
    (20, 'feature_46'): -2.150476,
    (20, 'feature_47'): 0.490073,
    (20, 'feature_48'): -0.763451,
    (20, 'feature_49'): -0.171070,
    (20, 'feature_50'): 0.431658,
    (20, 'feature_51'): -0.384785,
    (20, 'feature_52'): -0.427143,
    (20, 'feature_53'): -1.389143,
    (20, 'feature_54'): -0.224384,
    (20, 'feature_55'): -0.444930,
    (20, 'feature_56'): 1.271850,
    (20, 'feature_57'): -0.536142,
    (20, 'feature_58'): -0.236356,
    (20, 'feature_59'): -1.313975,
    (20, 'feature_60'): -0.433162,
    (20, 'feature_61'): 0.845812,
    (20, 'feature_62'): -0.309505,
    (20, 'feature_63'): -0.351364,
    (20, 'feature_64'): -0.313265,
    (20, 'feature_65'): 0.353055,
    (20, 'feature_66'): 0.609334,
    (20, 'feature_67'): 0.557346,
    (20, 'feature_68'): 0.158204,
    (20, 'feature_69'): 0.718751,
    (20, 'feature_70'): 0.845456,
    (20, 'feature_71'): 0.002883,
    (20, 'feature_72'): 0.391985,
    (20, 'feature_73'): 2.082426,
    (20, 'feature_74'): 1.995615,
    (20, 'feature_75'): 2.616426,
    (20, 'feature_76'): 3.381838,
    (20, 'feature_77'): 2.820068,
    (20, 'feature_78'): 2.973961,
    (21, 'feature_00'): 0.822731,
    (21, 'feature_01'): -1.715876,
    (21, 'feature_02'): 0.910317,
    (21, 'feature_03'): 0.613819,
    (21, 'feature_04'): -0.890816,
    (21, 'feature_05'): -1.040508,
    (21, 'feature_06'): -0.210921,
    (21, 'feature_07'): -1.026217,
    (21, 'feature_08'): -0.050462,
    (21, 'feature_09'): 34.000000,
    (21, 'feature_10'): 4.000000,
    (21, 'feature_11'): 214.000000,
    (21, 'feature_12'): 0.128217,
    (21, 'feature_13'): 0.311816,
    (21, 'feature_14'): 0.242436,
    (21, 'feature_15'): -0.373816,
    (21, 'feature_16'): -0.496355,
    (21, 'feature_17'): -0.667239,
    (21, 'feature_18'): 0.090343,
    (21, 'feature_19'): 0.769294,
    (21, 'feature_20'): -0.612963,
    (21, 'feature_21'): -0.055060,
    (21, 'feature_22'): -0.694589,
    (21, 'feature_23'): -0.172171,
    (21, 'feature_24'): 1.708375,
    (21, 'feature_25'): 1.388601,
    (21, 'feature_26'): -0.824903,
    (21, 'feature_27'): -1.330435,
    (21, 'feature_28'): -1.238031,
    (21, 'feature_29'): -0.597670,
    (21, 'feature_30'): -0.487317,
    (21, 'feature_31'): -0.117198,
    (21, 'feature_32'): 1.076208,
    (21, 'feature_33'): 0.787243,
    (21, 'feature_34'): 0.192805,
    (21, 'feature_35'): 0.631261,
    (21, 'feature_36'): 1.550952,
    (21, 'feature_37'): 0.044474,
    (21, 'feature_38'): 0.159499,
    (21, 'feature_39'): -0.088138,
    (21, 'feature_40'): 0.548948,
    (21, 'feature_41'): -0.455391,
    (21, 'feature_42'): 0.748640,
    (21, 'feature_43'): 0.469105,
    (21, 'feature_44'): -0.068083,
    (21, 'feature_45'): 0.708871,
    (21, 'feature_46'): 0.813784,
    (21, 'feature_47'): 0.611097,
    (21, 'feature_48'): -0.234233,
    (21, 'feature_49'): 0.193367,
    (21, 'feature_50'): -0.424562,
    (21, 'feature_51'): -1.185177,
    (21, 'feature_52'): -0.486495,
    (21, 'feature_53'): -0.499371,
    (21, 'feature_54'): -0.566004,
    (21, 'feature_55'): -0.859620,
    (21, 'feature_56'): 0.511017,
    (21, 'feature_57'): -0.327500,
    (21, 'feature_58'): 0.532043,
    (21, 'feature_59'): -0.363236,
    (21, 'feature_60'): -0.028774,
    (21, 'feature_61'): 0.845812,
    (21, 'feature_62'): -0.467022,
    (21, 'feature_63'): -0.227286,
    (21, 'feature_64'): -0.309424,
    (21, 'feature_65'): -0.415227,
    (21, 'feature_66'): 0.410243,
    (21, 'feature_67'): -0.123639,
    (21, 'feature_68'): -0.116115,
    (21, 'feature_69'): -0.058357,
    (21, 'feature_70'): 0.407086,
    (21, 'feature_71'): 0.486914,
    (21, 'feature_72'): 0.768864,
    (21, 'feature_73'): 0.309957,
    (21, 'feature_74'): 0.225641,
    (21, 'feature_75'): 1.576764,
    (21, 'feature_76'): 1.429681,
    (21, 'feature_77'): 0.723624,
    (21, 'feature_78'): 0.431078,
    (22, 'feature_00'): 0.979285,
    (22, 'feature_01'): -1.077667,
    (22, 'feature_02'): 0.222637,
    (22, 'feature_03'): 0.227630,
    (22, 'feature_04'): -0.838561,
    (22, 'feature_05'): -1.620982,
    (22, 'feature_06'): -0.265488,
    (22, 'feature_07'): -1.930789,
    (22, 'feature_08'): -0.056801,
    (22, 'feature_09'): 30.000000,
    (22, 'feature_10'): 10.000000,
    (22, 'feature_11'): 13.000000,
    (22, 'feature_12'): 0.005766,
    (22, 'feature_13'): -0.110832,
    (22, 'feature_14'): -0.096022,
    (22, 'feature_15'): -0.580600,
    (22, 'feature_16'): -0.582094,
    (22, 'feature_17'): -0.317458,
    (22, 'feature_18'): -0.053218,
    (22, 'feature_19'): 0.210845,
    (22, 'feature_20'): -0.162534,
    (22, 'feature_21'): -0.207907,
    (22, 'feature_22'): -0.990980,
    (22, 'feature_23'): -0.268050,
    (22, 'feature_24'): 0.542853,
    (22, 'feature_25'): -0.369906,
    (22, 'feature_26'): 0.300850,
    (22, 'feature_27'): -0.782905,
    (22, 'feature_28'): -1.151000,
    (22, 'feature_29'): -0.483665,
    (22, 'feature_30'): -0.700576,
    (22, 'feature_31'): -0.185681,
    (22, 'feature_32'): 0.835596,
    (22, 'feature_33'): 0.086263,
    (22, 'feature_34'): 1.037753,
    (22, 'feature_35'): 1.005987,
    (22, 'feature_36'): 0.368294,
    (22, 'feature_37'): -0.977194,
    (22, 'feature_38'): -1.085860,
    (22, 'feature_39'): 0.308925,
    (22, 'feature_40'): 0.075474,
    (22, 'feature_41'): -0.044268,
    (22, 'feature_42'): 0.387344,
    (22, 'feature_43'): -0.685271,
    (22, 'feature_44'): -0.665944,
    (22, 'feature_45'): 1.074115,
    (22, 'feature_46'): 0.874277,
    (22, 'feature_47'): 0.040311,
    (22, 'feature_48'): 0.418490,
    (22, 'feature_49'): 0.506215,
    (22, 'feature_50'): 0.361441,
    (22, 'feature_51'): -0.243091,
    (22, 'feature_52'): 0.405749,
    (22, 'feature_53'): 0.658526,
    (22, 'feature_54'): 0.093297,
    (22, 'feature_55'): -0.698255,
    (22, 'feature_56'): 0.482289,
    (22, 'feature_57'): 1.104285,
    (22, 'feature_58'): 0.103283,
    (22, 'feature_59'): 0.924651,
    (22, 'feature_60'): 0.398196,
    (22, 'feature_61'): 0.845812,
    (22, 'feature_62'): -0.411626,
    (22, 'feature_63'): -0.418186,
    (22, 'feature_64'): -0.346224,
    (22, 'feature_65'): 0.099692,
    (22, 'feature_66'): 2.216896,
    (22, 'feature_67'): 0.038209,
    (22, 'feature_68'): -0.175390,
    (22, 'feature_69'): -0.036280,
    (22, 'feature_70'): -0.013935,
    (22, 'feature_71'): -0.040513,
    (22, 'feature_72'): -0.151310,
    (22, 'feature_73'): -0.211425,
    (22, 'feature_74'): -0.110781,
    (22, 'feature_75'): -0.163626,
    (22, 'feature_76'): -0.191782,
    (22, 'feature_77'): -0.185764,
    (22, 'feature_78'): -0.064962,
    (23, 'feature_00'): 1.103988,
    (23, 'feature_01'): -1.409838,
    (23, 'feature_02'): 0.679237,
    (23, 'feature_03'): 0.271988,
    (23, 'feature_04'): -0.921083,
    (23, 'feature_05'): -1.655350,
    (23, 'feature_06'): -0.333362,
    (23, 'feature_07'): -0.847898,
    (23, 'feature_08'): -0.128456,
    (23, 'feature_09'): 34.000000,
    (23, 'feature_10'): 4.000000,
    (23, 'feature_11'): 214.000000,
    (23, 'feature_12'): 0.067833,
    (23, 'feature_13'): -0.149095,
    (23, 'feature_14'): 0.389475,
    (23, 'feature_15'): -0.704847,
    (23, 'feature_16'): -0.187946,
    (23, 'feature_17'): -0.470747,
    (23, 'feature_18'): 0.385676,
    (23, 'feature_19'): 1.928927,
    (23, 'feature_20'): -0.280370,
    (23, 'feature_21'): -0.181847,
    (23, 'feature_22'): 0.560017,
    (23, 'feature_23'): 0.542165,
    (23, 'feature_24'): 0.420537,
    (23, 'feature_25'): -1.093146,
    (23, 'feature_26'): 1.263360,
    (23, 'feature_27'): 0.721569,
    (23, 'feature_28'): 0.227230,
    (23, 'feature_29'): -0.823615,
    (23, 'feature_30'): -0.490956,
    (23, 'feature_31'): -0.143314,
    (23, 'feature_32'): 0.722424,
    (23, 'feature_33'): -0.201024,
    (23, 'feature_34'): 1.398427,
    (23, 'feature_35'): 1.535538,
    (23, 'feature_36'): -1.203048,
    (23, 'feature_37'): -0.062877,
    (23, 'feature_38'): 0.365412,
    (23, 'feature_39'): -0.433845,
    (23, 'feature_40'): 0.101690,
    (23, 'feature_41'): 0.880660,
    (23, 'feature_42'): -1.147863,
    (23, 'feature_43'): -0.519921,
    (23, 'feature_44'): -0.826520,
    (23, 'feature_45'): 0.208618,
    (23, 'feature_46'): -0.206861,
    (23, 'feature_47'): -0.243556,
    (23, 'feature_48'): 0.187892,
    (23, 'feature_49'): -0.445879,
    (23, 'feature_50'): -0.688431,
    (23, 'feature_51'): 0.981535,
    (23, 'feature_52'): 0.628313,
    (23, 'feature_53'): -0.893709,
    (23, 'feature_54'): -0.222755,
    (23, 'feature_55'): 0.022181,
    (23, 'feature_56'): 0.171491,
    (23, 'feature_57'): 0.140035,
    (23, 'feature_58'): -0.454531,
    (23, 'feature_59'): 0.432669,
    (23, 'feature_60'): -0.209559,
    (23, 'feature_61'): 0.845812,
    (23, 'feature_62'): -0.149235,
    (23, 'feature_63'): -0.163416,
    (23, 'feature_64'): -0.406172,
    (23, 'feature_65'): -0.526801,
    (23, 'feature_66'): 2.264564,
    (23, 'feature_67'): 0.055226,
    (23, 'feature_68'): -0.207911,
    (23, 'feature_69'): 0.423878,
    (23, 'feature_70'): 0.151445,
    (23, 'feature_71'): -0.035186,
    (23, 'feature_72'): 0.308554,
    (23, 'feature_73'): -0.180651,
    (23, 'feature_74'): -0.235350,
    (23, 'feature_75'): 0.377031,
    (23, 'feature_76'): 0.289521,
    (23, 'feature_77'): -0.029421,
    (23, 'feature_78'): -0.049873,
    (24, 'feature_00'): -0.232035,
    (24, 'feature_01'): -1.902407,
    (24, 'feature_02'): 0.617699,
    (24, 'feature_03'): 0.133120,
    (24, 'feature_04'): -0.940508,
    (24, 'feature_05'): -1.216277,
    (24, 'feature_06'): -0.160162,
    (24, 'feature_07'): -1.375872,
    (24, 'feature_08'): -0.136182,
    (24, 'feature_09'): 14.000000,
    (24, 'feature_10'): 10.000000,
    (24, 'feature_11'): 40.000000,
    (24, 'feature_12'): 0.165189,
    (24, 'feature_13'): 0.756379,
    (24, 'feature_14'): 0.889424,
    (24, 'feature_15'): -0.608788,
    (24, 'feature_16'): -0.591084,
    (24, 'feature_17'): -0.307653,
    (24, 'feature_18'): 0.426028,
    (24, 'feature_19'): 0.818275,
    (24, 'feature_20'): 0.396884,
    (24, 'feature_21'): -0.197326,
    (24, 'feature_22'): -0.727286,
    (24, 'feature_23'): -0.321978,
    (24, 'feature_24'): 1.425789,
    (24, 'feature_25'): 1.260155,
    (24, 'feature_26'): -0.935433,
    (24, 'feature_27'): -1.306859,
    (24, 'feature_28'): -0.870019,
    (24, 'feature_29'): -0.413003,
    (24, 'feature_30'): -0.746992,
    (24, 'feature_31'): -0.199736,
    (24, 'feature_32'): 0.710212,
    (24, 'feature_33'): 0.303655,
    (24, 'feature_34'): 1.167775,
    (24, 'feature_35'): 0.863421,
    (24, 'feature_36'): 0.665224,
    (24, 'feature_37'): -0.114164,
    (24, 'feature_38'): -0.466461,
    (24, 'feature_39'): 0.292654,
    (24, 'feature_40'): 0.787276,
    (24, 'feature_41'): 0.576979,
    (24, 'feature_42'): -0.761330,
    (24, 'feature_43'): 1.242574,
    (24, 'feature_44'): 0.504388,
    (24, 'feature_45'): 0.465723,
    (24, 'feature_46'): -0.805513,
    (24, 'feature_47'): -0.243884,
    (24, 'feature_48'): 0.307793,
    (24, 'feature_49'): 0.308907,
    (24, 'feature_50'): 0.768484,
    (24, 'feature_51'): -1.079759,
    (24, 'feature_52'): 1.026576,
    (24, 'feature_53'): -0.899987,
    (24, 'feature_54'): -0.109521,
    (24, 'feature_55'): 1.208506,
    (24, 'feature_56'): 1.023236,
    (24, 'feature_57'): -0.631863,
    (24, 'feature_58'): -0.676444,
    (24, 'feature_59'): 0.237457,
    (24, 'feature_60'): 0.064384,
    (24, 'feature_61'): 0.845812,
    (24, 'feature_62'): -0.442071,
    (24, 'feature_63'): -0.294408,
    (24, 'feature_64'): -0.460089,
    (24, 'feature_65'): 0.817921,
    (24, 'feature_66'): 0.722203,
    (24, 'feature_67'): 0.108435,
    (24, 'feature_68'): 0.078961,
    (24, 'feature_69'): 0.281010,
    (24, 'feature_70'): 0.283387,
    (24, 'feature_71'): 1.235469,
    (24, 'feature_72'): 1.094170,
    (24, 'feature_73'): 0.162002,
    (24, 'feature_74'): 0.153789,
    (24, 'feature_75'): -0.075862,
    (24, 'feature_76'): -0.045521,
    (24, 'feature_77'): -0.043211,
    (24, 'feature_78'): 0.003116,
    (25, 'feature_00'): 0.456349,
    (25, 'feature_01'): -1.300463,
    (25, 'feature_02'): 0.328758,
    (25, 'feature_03'): 0.757183,
    (25, 'feature_04'): -1.307577,
    (25, 'feature_05'): -1.306825,
    (25, 'feature_06'): -0.321634,
    (25, 'feature_07'): -1.267030,
    (25, 'feature_08'): -0.070750,
    (25, 'feature_09'): 57.000000,
    (25, 'feature_10'): 1.000000,
    (25, 'feature_11'): 230.000000,
    (25, 'feature_12'): -0.093449,
    (25, 'feature_13'): 1.060057,
    (25, 'feature_14'): 0.312515,
    (25, 'feature_15'): -0.505854,
    (25, 'feature_16'): -0.480670,
    (25, 'feature_17'): -0.344321,
    (25, 'feature_18'): 0.375060,
    (25, 'feature_19'): 3.520319,
    (25, 'feature_20'): 0.578240,
    (25, 'feature_21'): 0.006407,
    (25, 'feature_22'): 0.240094,
    (25, 'feature_23'): 0.610826,
    (25, 'feature_24'): 1.156997,
    (25, 'feature_25'): 0.045539,
    (25, 'feature_26'): -0.631373,
    (25, 'feature_27'): -0.665180,
    (25, 'feature_28'): -0.751228,
    (25, 'feature_29'): -0.452564,
    (25, 'feature_30'): -0.751713,
    (25, 'feature_31'): 0.005909,
    (25, 'feature_32'): 1.134045,
    (25, 'feature_33'): -0.000379,
    (25, 'feature_34'): 1.167570,
    (25, 'feature_35'): 1.236236,
    (25, 'feature_36'): 0.806501,
    (25, 'feature_37'): -0.618816,
    (25, 'feature_38'): -0.865145,
    (25, 'feature_39'): 0.312555,
    (25, 'feature_40'): 1.201090,
    (25, 'feature_41'): -0.127505,
    (25, 'feature_42'): 0.205120,
    (25, 'feature_43'): -1.009632,
    (25, 'feature_44'): -0.580314,
    (25, 'feature_45'): 1.660312,
    (25, 'feature_46'): 0.851810,
    (25, 'feature_47'): 0.414463,
    (25, 'feature_48'): -0.655579,
    (25, 'feature_49'): 0.243940,
    (25, 'feature_50'): 0.460581,
    (25, 'feature_51'): 0.207402,
    (25, 'feature_52'): 0.251171,
    (25, 'feature_53'): 0.534479,
    (25, 'feature_54'): -1.369717,
    (25, 'feature_55'): 0.250418,
    (25, 'feature_56'): 1.259639,
    (25, 'feature_57'): 1.166466,
    (25, 'feature_58'): 0.719141,
    (25, 'feature_59'): -1.090652,
    (25, 'feature_60'): 0.265632,
    (25, 'feature_61'): 0.845812,
    (25, 'feature_62'): -0.448936,
    (25, 'feature_63'): -0.219867,
    (25, 'feature_64'): -0.212735,
    (25, 'feature_65'): 0.943353,
    (25, 'feature_66'): 2.385009,
    (25, 'feature_67'): -0.252938,
    (25, 'feature_68'): 0.764376,
    (25, 'feature_69'): 0.214034,
    (25, 'feature_70'): 0.012668,
    (25, 'feature_71'): 0.662211,
    (25, 'feature_72'): 0.566757,
    (25, 'feature_73'): -0.125968,
    (25, 'feature_74'): -0.062968,
    (25, 'feature_75'): 0.556956,
    (25, 'feature_76'): 0.433440,
    (25, 'feature_77'): -0.084712,
    (25, 'feature_78'): -0.027083,
    (26, 'feature_00'): 0.273828,
    (26, 'feature_01'): -1.201698,
    (26, 'feature_02'): 0.790333,
    (26, 'feature_03'): 0.502330,
    (26, 'feature_04'): -0.401575,
    (26, 'feature_05'): -1.583591,
    (26, 'feature_06'): -0.213574,
    (26, 'feature_07'): -0.906497,
    (26, 'feature_08'): -0.155891,
    (26, 'feature_09'): 26.000000,
    (26, 'feature_10'): 12.000000,
    (26, 'feature_11'): 158.000000,
    (26, 'feature_12'): -0.029102,
    (26, 'feature_13'): 0.198433,
    (26, 'feature_14'): 0.041310,
    (26, 'feature_15'): -1.039926,
    (26, 'feature_16'): -0.619869,
    (26, 'feature_17'): -0.809292,
    (26, 'feature_18'): 0.186474,
    (26, 'feature_19'): 2.881960,
    (26, 'feature_20'): -0.649529,
    (26, 'feature_21'): -0.113812,
    (26, 'feature_22'): -0.964969,
    (26, 'feature_23'): -0.789987,
    (26, 'feature_24'): 0.540473,
    (26, 'feature_25'): -0.165649,
    (26, 'feature_26'): -2.008217,
    (26, 'feature_27'): -1.714908,
    (26, 'feature_28'): -1.020191,
    (26, 'feature_29'): -0.672271,
    (26, 'feature_30'): -0.877176,
    (26, 'feature_31'): -0.149161,
    (26, 'feature_32'): 0.973723,
    (26, 'feature_33'): -0.042592,
    (26, 'feature_34'): 1.192620,
    (26, 'feature_35'): 0.949918,
    (26, 'feature_36'): 0.347720,
    (26, 'feature_37'): -0.013147,
    (26, 'feature_38'): -0.081949,
    (26, 'feature_39'): 0.255850,
    (26, 'feature_40'): -0.543102,
    (26, 'feature_41'): -0.293992,
    (26, 'feature_42'): -0.431497,
    (26, 'feature_43'): -0.164019,
    (26, 'feature_44'): -0.035206,
    (26, 'feature_45'): 0.976596,
    (26, 'feature_46'): 0.228540,
    (26, 'feature_47'): 0.257156,
    (26, 'feature_48'): -0.330632,
    (26, 'feature_49'): -0.225638,
    (26, 'feature_50'): -0.501238,
    (26, 'feature_51'): -0.643585,
    (26, 'feature_52'): -1.536210,
    (26, 'feature_53'): -0.463512,
    (26, 'feature_54'): -1.190573,
    (26, 'feature_55'): -0.406502,
    (26, 'feature_56'): 0.195491,
    (26, 'feature_57'): 0.439230,
    (26, 'feature_58'): 0.255838,
    (26, 'feature_59'): -0.921487,
    (26, 'feature_60'): -0.531326,
    (26, 'feature_61'): 0.845812,
    (26, 'feature_62'): -0.484515,
    (26, 'feature_63'): -0.280161,
    (26, 'feature_64'): -0.364524,
    (26, 'feature_65'): 0.190260,
    (26, 'feature_66'): 2.079438,
    (26, 'feature_67'): -0.033414,
    (26, 'feature_68'): 0.395012,
    (26, 'feature_69'): 0.047707,
    (26, 'feature_70'): -0.069487,
    (26, 'feature_71'): -0.075117,
    (26, 'feature_72'): 0.029779,
    (26, 'feature_73'): 0.056931,
    (26, 'feature_74'): 0.070900,
    (26, 'feature_75'): 1.622438,
    (26, 'feature_76'): 1.985820,
    (26, 'feature_77'): 0.422320,
    (26, 'feature_78'): 0.331347,
    (27, 'feature_00'): 0.604257,
    (27, 'feature_01'): -1.562618,
    (27, 'feature_02'): 1.123856,
    (27, 'feature_03'): 0.475217,
    (27, 'feature_04'): -0.973835,
    (27, 'feature_05'): -1.145395,
    (27, 'feature_06'): -0.169014,
    (27, 'feature_07'): -0.879498,
    (27, 'feature_08'): -0.132850,
    (27, 'feature_09'): 82.000000,
    (27, 'feature_10'): 3.000000,
    (27, 'feature_11'): 539.000000,
    (27, 'feature_12'): -0.280626,
    (27, 'feature_13'): 0.364261,
    (27, 'feature_14'): 0.123539,
    (27, 'feature_15'): -0.756725,
    (27, 'feature_16'): -0.710923,
    (27, 'feature_17'): -0.548767,
    (27, 'feature_18'): -0.324842,
    (27, 'feature_19'): 3.334690,
    (27, 'feature_20'): 0.086138,
    (27, 'feature_21'): -0.152199,
    (27, 'feature_22'): 0.879677,
    (27, 'feature_23'): 0.223121,
    (27, 'feature_24'): 3.114479,
    (27, 'feature_25'): 1.490302,
    (27, 'feature_26'): -1.139511,
    (27, 'feature_27'): -0.130644,
    (27, 'feature_28'): 0.455322,
    (27, 'feature_29'): -0.802515,
    (27, 'feature_30'): -0.937626,
    (27, 'feature_31'): -0.075217,
    (27, 'feature_32'): -0.140108,
    (27, 'feature_33'): -1.812494,
    (27, 'feature_34'): 0.237969,
    (27, 'feature_35'): 0.165008,
    (27, 'feature_36'): -0.630636,
    (27, 'feature_37'): -1.040017,
    (27, 'feature_38'): -0.130328,
    (27, 'feature_39'): 0.112575,
    (27, 'feature_40'): 0.018146,
    (27, 'feature_41'): -0.131169,
    (27, 'feature_42'): 0.057425,
    (27, 'feature_43'): -0.769814,
    (27, 'feature_44'): -0.748363,
    (27, 'feature_45'): 0.697195,
    (27, 'feature_46'): 0.118134,
    (27, 'feature_47'): -0.106042,
    (27, 'feature_48'): -0.316969,
    (27, 'feature_49'): 0.037821,
    (27, 'feature_50'): 0.443382,
    (27, 'feature_51'): -0.701233,
    (27, 'feature_52'): -0.653942,
    (27, 'feature_53'): 0.909394,
    (27, 'feature_54'): -0.138249,
    (27, 'feature_55'): 0.285082,
    (27, 'feature_56'): 0.711658,
    (27, 'feature_57'): 2.047650,
    (27, 'feature_58'): 0.300342,
    (27, 'feature_59'): -0.479424,
    (27, 'feature_60'): 0.230571,
    (27, 'feature_61'): 0.845812,
    (27, 'feature_62'): -0.532152,
    (27, 'feature_63'): -0.280711,
    (27, 'feature_64'): -0.403614,
    (27, 'feature_65'): -0.344646,
    (27, 'feature_66'): 1.130334,
    (27, 'feature_67'): -0.261155,
    (27, 'feature_68'): 0.549332,
    (27, 'feature_69'): 0.082389,
    (27, 'feature_70'): -0.133721,
    (27, 'feature_71'): 0.036189,
    (27, 'feature_72'): 0.079372,
    (27, 'feature_73'): -0.519674,
    (27, 'feature_74'): -0.440173,
    (27, 'feature_75'): -0.224271,
    (27, 'feature_76'): -0.208948,
    (27, 'feature_77'): -0.495665,
    (27, 'feature_78'): -0.582771,
    (28, 'feature_00'): 0.370953,
    (28, 'feature_01'): -1.720037,
    (28, 'feature_02'): 0.453694,
    (28, 'feature_03'): 0.153077,
    (28, 'feature_04'): -0.918634,
    (28, 'feature_05'): -1.263774,
    (28, 'feature_06'): -0.379116,
    (28, 'feature_07'): -1.374199,
    (28, 'feature_08'): -0.157073,
    (28, 'feature_09'): 9.000000,
    (28, 'feature_10'): 3.000000,
    (28, 'feature_11'): 25.000000,
    (28, 'feature_12'): -0.075194,
    (28, 'feature_13'): -0.227188,
    (28, 'feature_14'): -0.217686,
    (28, 'feature_15'): -0.734078,
    (28, 'feature_16'): -0.397530,
    (28, 'feature_17'): -0.602520,
    (28, 'feature_18'): -0.072593,
    (28, 'feature_19'): 1.632876,
    (28, 'feature_20'): -0.287741,
    (28, 'feature_21'): -0.214352,
    (28, 'feature_22'): -0.237557,
    (28, 'feature_23'): -0.061020,
    (28, 'feature_24'): 0.583718,
    (28, 'feature_25'): -0.084847,
    (28, 'feature_26'): -1.233504,
    (28, 'feature_27'): -0.230320,
    (28, 'feature_28'): 1.137674,
    (28, 'feature_29'): -0.570627,
    (28, 'feature_30'): -0.696661,
    (28, 'feature_31'): -0.212154,
    (28, 'feature_32'): 1.404737,
    (28, 'feature_33'): -0.602051,
    (28, 'feature_34'): 0.960254,
    (28, 'feature_35'): 0.982120,
    (28, 'feature_36'): -0.936638,
    (28, 'feature_37'): -0.304719,
    (28, 'feature_38'): -0.296530,
    (28, 'feature_39'): -0.388414,
    (28, 'feature_40'): 0.024696,
    (28, 'feature_41'): -0.708426,
    (28, 'feature_42'): -0.325782,
    (28, 'feature_43'): 0.119690,
    (28, 'feature_44'): -0.353069,
    (28, 'feature_45'): 0.865673,
    (28, 'feature_46'): -0.655430,
    (28, 'feature_47'): -0.335790,
    (28, 'feature_48'): -0.082408,
    (28, 'feature_49'): -0.173892,
    (28, 'feature_50'): 0.012471,
    (28, 'feature_51'): -0.443761,
    (28, 'feature_52'): -0.307534,
    (28, 'feature_53'): -0.346765,
    (28, 'feature_54'): 0.410051,
    (28, 'feature_55'): -0.679426,
    (28, 'feature_56'): -0.051368,
    (28, 'feature_57'): -0.562536,
    (28, 'feature_58'): -0.551760,
    (28, 'feature_59'): -0.237596,
    (28, 'feature_60'): -0.254276,
    (28, 'feature_61'): 0.845812,
    (28, 'feature_62'): -0.390331,
    (28, 'feature_63'): -0.175438,
    (28, 'feature_64'): -0.333701,
    (28, 'feature_65'): -0.496176,
    (28, 'feature_66'): 1.257112,
    (28, 'feature_67'): -0.035366,
    (28, 'feature_68'): -0.135724,
    (28, 'feature_69'): -0.094565,
    (28, 'feature_70'): -0.189035,
    (28, 'feature_71'): -0.223391,
    (28, 'feature_72'): -0.222060,
    (28, 'feature_73'): 0.274175,
    (28, 'feature_74'): 0.257223,
    (28, 'feature_75'): 0.042938,
    (28, 'feature_76'): 0.083366,
    (28, 'feature_77'): 0.865723,
    (28, 'feature_78'): 0.843362,
    (29, 'feature_00'): -0.039533,
    (29, 'feature_01'): -1.530588,
    (29, 'feature_02'): 0.668950,
    (29, 'feature_03'): 0.408206,
    (29, 'feature_04'): -0.857224,
    (29, 'feature_05'): -1.321894,
    (29, 'feature_06'): -0.160530,
    (29, 'feature_07'): -1.344891,
    (29, 'feature_08'): -0.143504,
    (29, 'feature_09'): 57.000000,
    (29, 'feature_10'): 1.000000,
    (29, 'feature_11'): 336.000000,
    (29, 'feature_12'): 0.959105,
    (29, 'feature_13'): 3.323562,
    (29, 'feature_14'): 1.746511,
    (29, 'feature_15'): -1.188930,
    (29, 'feature_16'): -0.591225,
    (29, 'feature_17'): -0.864866,
    (29, 'feature_18'): 0.070191,
    (29, 'feature_19'): 2.465837,
    (29, 'feature_20'): 0.496998,
    (29, 'feature_21'): -0.045664,
    (29, 'feature_22'): -0.616913,
    (29, 'feature_23'): -0.567811,
    (29, 'feature_24'): 1.350605,
    (29, 'feature_25'): 1.177551,
    (29, 'feature_26'): 0.194944,
    (29, 'feature_27'): -1.356428,
    (29, 'feature_28'): -1.488750,
    (29, 'feature_29'): -0.620147,
    (29, 'feature_30'): -0.982724,
    (29, 'feature_31'): -0.048024,
    (29, 'feature_32'): -0.069649,
    (29, 'feature_33'): -1.189370,
    (29, 'feature_34'): 0.799811,
    (29, 'feature_35'): 0.755163,
    (29, 'feature_36'): -1.576469,
    (29, 'feature_37'): -1.374968,
    (29, 'feature_38'): -1.636843,
    (29, 'feature_39'): -0.683494,
    (29, 'feature_40'): 0.207657,
    (29, 'feature_41'): -0.353415,
    (29, 'feature_42'): -0.169158,
    (29, 'feature_43'): -0.912512,
    (29, 'feature_44'): -0.624035,
    (29, 'feature_45'): 1.323134,
    (29, 'feature_46'): -0.321535,
    (29, 'feature_47'): -0.383160,
    (29, 'feature_48'): -1.160915,
    (29, 'feature_49'): -0.257920,
    (29, 'feature_50'): 1.067895,
    (29, 'feature_51'): -1.163018,
    (29, 'feature_52'): -0.742785,
    (29, 'feature_53'): -0.461696,
    (29, 'feature_54'): -1.925587,
    (29, 'feature_55'): -0.291952,
    (29, 'feature_56'): 1.565421,
    (29, 'feature_57'): 0.552652,
    (29, 'feature_58'): 0.280293,
    (29, 'feature_59'): -1.831175,
    (29, 'feature_60'): 0.010189,
    (29, 'feature_61'): 0.845812,
    (29, 'feature_62'): -0.357684,
    (29, 'feature_63'): -0.188879,
    (29, 'feature_64'): -0.264406,
    (29, 'feature_65'): -0.688957,
    (29, 'feature_66'): 1.165713,
    (29, 'feature_67'): 0.665102,
    (29, 'feature_68'): 1.862187,
    (29, 'feature_69'): 1.556595,
    (29, 'feature_70'): 0.672900,
    (29, 'feature_71'): 4.524442,
    (29, 'feature_72'): 1.569038,
    (29, 'feature_73'): -1.280046,
    (29, 'feature_74'): -0.833000,
    (29, 'feature_75'): -0.284026,
    (29, 'feature_76'): -0.146068,
    (29, 'feature_77'): -1.043098,
    (29, 'feature_78'): -1.065085,
    (30, 'feature_00'): 0.004876,
    (30, 'feature_01'): -1.797837,
    (30, 'feature_02'): 0.405377,
    (30, 'feature_03'): 0.451257,
    (30, 'feature_04'): -0.682259,
    (30, 'feature_05'): -0.868485,
    (30, 'feature_06'): -0.156585,
    (30, 'feature_07'): -1.058445,
    (30, 'feature_08'): -0.156712,
    (30, 'feature_09'): 81.000000,
    (30, 'feature_10'): 2.000000,
    (30, 'feature_11'): 534.000000,
    (30, 'feature_12'): -0.181802,
    (30, 'feature_13'): -0.066491,
    (30, 'feature_14'): 0.170403,
    (30, 'feature_15'): -0.572133,
    (30, 'feature_16'): -0.598495,
    (30, 'feature_17'): -0.653849,
    (30, 'feature_18'): 1.035732,
    (30, 'feature_19'): -1.532879,
    (30, 'feature_20'): 0.465593,
    (30, 'feature_21'): 0.293331,
    (30, 'feature_22'): 0.247911,
    (30, 'feature_23'): -0.030064,
    (30, 'feature_24'): 1.836518,
    (30, 'feature_25'): 1.281834,
    (30, 'feature_26'): -1.612357,
    (30, 'feature_27'): -1.231617,
    (30, 'feature_28'): -0.529558,
    (30, 'feature_29'): -0.533356,
    (30, 'feature_30'): -0.755730,
    (30, 'feature_31'): 0.290029,
    (30, 'feature_32'): 0.970692,
    (30, 'feature_33'): -0.219509,
    (30, 'feature_34'): 0.934300,
    (30, 'feature_35'): 1.256169,
    (30, 'feature_36'): -0.060696,
    (30, 'feature_37'): 0.634269,
    (30, 'feature_38'): 0.439075,
    (30, 'feature_39'): 0.936693,
    (30, 'feature_40'): 1.660053,
    (30, 'feature_41'): 1.197685,
    (30, 'feature_42'): -1.427574,
    (30, 'feature_43'): 0.121745,
    (30, 'feature_44'): 0.032295,
    (30, 'feature_45'): 0.465514,
    (30, 'feature_46'): 0.125376,
    (30, 'feature_47'): -0.084060,
    (30, 'feature_48'): 4.436279,
    (30, 'feature_49'): 0.017492,
    (30, 'feature_50'): 0.208398,
    (30, 'feature_51'): 1.765119,
    (30, 'feature_52'): 1.175174,
    (30, 'feature_53'): -0.602604,
    (30, 'feature_54'): 0.848583,
    (30, 'feature_55'): 1.858203,
    (30, 'feature_56'): 0.959057,
    (30, 'feature_57'): -0.325480,
    (30, 'feature_58'): -0.089357,
    (30, 'feature_59'): 2.774514,
    (30, 'feature_60'): 0.017130,
    (30, 'feature_61'): 0.845812,
    (30, 'feature_62'): -0.485199,
    (30, 'feature_63'): -0.299220,
    (30, 'feature_64'): -0.389775,
    (30, 'feature_65'): 1.219949,
    (30, 'feature_66'): -2.283324,
    (30, 'feature_67'): -0.172930,
    (30, 'feature_68'): -0.035929,
    (30, 'feature_69'): 0.152410,
    (30, 'feature_70'): -0.197636,
    (30, 'feature_71'): -0.124129,
    (30, 'feature_72'): -0.121228,
    (30, 'feature_73'): -0.329773,
    (30, 'feature_74'): -0.475042,
    (30, 'feature_75'): 0.138360,
    (30, 'feature_76'): 0.047495,
    (30, 'feature_77'): -0.184340,
    (30, 'feature_78'): -0.240573,
    (31, 'feature_00'): 0.952687,
    (31, 'feature_01'): -2.076453,
    (31, 'feature_02'): 1.079939,
    (31, 'feature_03'): 0.431252,
    (31, 'feature_04'): -0.991194,
    (31, 'feature_05'): -1.133483,
    (31, 'feature_06'): -0.237640,
    (31, 'feature_07'): -1.306140,
    (31, 'feature_08'): -0.092842,
    (31, 'feature_09'): 44.000000,
    (31, 'feature_10'): 3.000000,
    (31, 'feature_11'): 24.000000,
    (31, 'feature_12'): 0.087379,
    (31, 'feature_13'): 0.157700,
    (31, 'feature_14'): 0.424698,
    (31, 'feature_15'): -0.994068,
    (31, 'feature_16'): -0.790850,
    (31, 'feature_17'): -0.731936,
    (31, 'feature_18'): 0.207778,
    (31, 'feature_19'): 1.577352,
    (31, 'feature_20'): 0.251641,
    (31, 'feature_21'): -0.204461,
    (31, 'feature_22'): -1.467595,
    (31, 'feature_23'): -0.713402,
    (31, 'feature_24'): 1.575408,
    (31, 'feature_25'): 1.402582,
    (31, 'feature_26'): 0.135072,
    (31, 'feature_27'): -1.231982,
    (31, 'feature_28'): -1.966784,
    (31, 'feature_29'): -0.739180,
    (31, 'feature_30'): -0.754005,
    (31, 'feature_31'): -0.128405,
    (31, 'feature_32'): -0.115075,
    (31, 'feature_33'): -0.701563,
    (31, 'feature_34'): 0.388347,
    (31, 'feature_35'): -0.231645,
    (31, 'feature_36'): -0.333558,
    (31, 'feature_37'): -0.573108,
    (31, 'feature_38'): -0.600406,
    (31, 'feature_39'): 0.179141,
    (31, 'feature_40'): 0.516064,
    (31, 'feature_41'): 0.503990,
    (31, 'feature_42'): -1.880085,
    (31, 'feature_43'): 0.527643,
    (31, 'feature_44'): -0.012222,
    (31, 'feature_45'): 1.628170,
    (31, 'feature_46'): -1.227695,
    (31, 'feature_47'): -0.318916,
    (31, 'feature_48'): 0.539196,
    (31, 'feature_49'): -0.109119,
    (31, 'feature_50'): -0.040619,
    (31, 'feature_51'): 0.626758,
    (31, 'feature_52'): 0.280940,
    (31, 'feature_53'): -0.600465,
    (31, 'feature_54'): -0.350392,
    (31, 'feature_55'): 0.712820,
    (31, 'feature_56'): 0.926478,
    (31, 'feature_57'): -0.435107,
    (31, 'feature_58'): -0.391214,
    (31, 'feature_59'): 0.058474,
    (31, 'feature_60'): -0.354109,
    (31, 'feature_61'): 0.845812,
    (31, 'feature_62'): -0.475799,
    (31, 'feature_63'): -0.313140,
    (31, 'feature_64'): -0.303393,
    (31, 'feature_65'): 0.132542,
    (31, 'feature_66'): 0.866975,
    (31, 'feature_67'): 0.155403,
    (31, 'feature_68'): 0.231855,
    (31, 'feature_69'): 0.602903,
    (31, 'feature_70'): 0.083362,
    (31, 'feature_71'): -0.025937,
    (31, 'feature_72'): 0.487009,
    (31, 'feature_73'): -2.360141,
    (31, 'feature_74'): -2.811445,
    (31, 'feature_75'): -1.359334,
    (31, 'feature_76'): -2.153715,
    (31, 'feature_77'): -2.437798,
    (31, 'feature_78'): -1.465698,
    (32, 'feature_00'): -0.143738,
    (32, 'feature_01'): -1.061421,
    (32, 'feature_02'): -0.103665,
    (32, 'feature_03'): 0.111096,
    (32, 'feature_04'): -0.811887,
    (32, 'feature_05'): -1.169498,
    (32, 'feature_06'): -0.214482,
    (32, 'feature_07'): -1.053012,
    (32, 'feature_08'): -0.146240,
    (32, 'feature_09'): 26.000000,
    (32, 'feature_10'): 12.000000,
    (32, 'feature_11'): 158.000000,
    (32, 'feature_12'): 0.136727,
    (32, 'feature_13'): 0.342246,
    (32, 'feature_14'): 0.363447,
    (32, 'feature_15'): -0.690501,
    (32, 'feature_16'): -0.569212,
    (32, 'feature_17'): -0.657325,
    (32, 'feature_18'): -0.955809,
    (32, 'feature_19'): 1.741455,
    (32, 'feature_20'): 0.107363,
    (32, 'feature_21'): 0.093804,
    (32, 'feature_22'): -0.263217,
    (32, 'feature_23'): 0.490985,
    (32, 'feature_24'): 2.519762,
    (32, 'feature_25'): 1.582061,
    (32, 'feature_26'): 2.394868,
    (32, 'feature_27'): -1.838428,
    (32, 'feature_28'): -1.970938,
    (32, 'feature_29'): -0.440869,
    (32, 'feature_30'): -0.444765,
    (32, 'feature_31'): 0.144404,
    (32, 'feature_32'): 1.079781,
    (32, 'feature_33'): -0.294430,
    (32, 'feature_34'): 0.887089,
    (32, 'feature_35'): 1.275491,
    (32, 'feature_36'): -1.427302,
    (32, 'feature_37'): -1.082772,
    (32, 'feature_38'): -1.064631,
    (32, 'feature_39'): -0.295076,
    (32, 'feature_40'): -0.552956,
    (32, 'feature_41'): 0.867012,
    (32, 'feature_42'): -0.657999,
    (32, 'feature_43'): -0.546308,
    (32, 'feature_44'): 0.606933,
    (32, 'feature_45'): 0.973648,
    (32, 'feature_46'): 0.894859,
    (32, 'feature_47'): -0.186042,
    (32, 'feature_48'): -0.491013,
    (32, 'feature_49'): 0.037383,
    (32, 'feature_50'): 0.151609,
    (32, 'feature_51'): -0.074348,
    (32, 'feature_52'): 1.127758,
    (32, 'feature_53'): -0.254527,
    (32, 'feature_54'): -0.086219,
    (32, 'feature_55'): 0.782899,
    (32, 'feature_56'): 0.806606,
    (32, 'feature_57'): 0.718859,
    (32, 'feature_58'): -0.084129,
    (32, 'feature_59'): 0.155852,
    (32, 'feature_60'): 0.426852,
    (32, 'feature_61'): 0.845812,
    (32, 'feature_62'): -0.405409,
    (32, 'feature_63'): -0.343199,
    (32, 'feature_64'): -0.385486,
    (32, 'feature_65'): -1.293582,
    (32, 'feature_66'): 2.000215,
    (32, 'feature_67'): 0.165290,
    (32, 'feature_68'): 0.101541,
    (32, 'feature_69'): 0.255502,
    (32, 'feature_70'): 0.035922,
    (32, 'feature_71'): 0.381999,
    (32, 'feature_72'): 0.354410,
    (32, 'feature_73'): -0.151174,
    (32, 'feature_74'): -0.013670,
    (32, 'feature_75'): -0.176047,
    (32, 'feature_76'): -0.019497,
    (32, 'feature_77'): -0.170170,
    (32, 'feature_78'): -0.067788,
    (33, 'feature_00'): 0.837706,
    (33, 'feature_01'): -1.378307,
    (33, 'feature_02'): 0.390039,
    (33, 'feature_03'): 0.383452,
    (33, 'feature_04'): -0.528775,
    (33, 'feature_05'): -1.718854,
    (33, 'feature_06'): -0.138896,
    (33, 'feature_07'): -1.116310,
    (33, 'feature_08'): -0.201308,
    (33, 'feature_09'): 11.000000,
    (33, 'feature_10'): 7.000000,
    (33, 'feature_11'): 76.000000,
    (33, 'feature_12'): 0.519016,
    (33, 'feature_13'): 1.604092,
    (33, 'feature_14'): 1.865379,
    (33, 'feature_15'): -0.673707,
    (33, 'feature_16'): -0.254448,
    (33, 'feature_17'): -0.350325,
    (33, 'feature_18'): 0.466579,
    (33, 'feature_19'): 2.231756,
    (33, 'feature_20'): 0.015520,
    (33, 'feature_21'): -0.302429,
    (33, 'feature_22'): -0.462090,
    (33, 'feature_23'): -0.266674,
    (33, 'feature_24'): 1.116082,
    (33, 'feature_25'): 0.224316,
    (33, 'feature_26'): -0.143123,
    (33, 'feature_27'): -0.312481,
    (33, 'feature_28'): -0.187941,
    (33, 'feature_29'): -0.623306,
    (33, 'feature_30'): -1.004519,
    (33, 'feature_31'): -0.300864,
    (33, 'feature_32'): 0.959138,
    (33, 'feature_33'): 0.009866,
    (33, 'feature_34'): 1.218436,
    (33, 'feature_35'): 0.878815,
    (33, 'feature_36'): 1.452894,
    (33, 'feature_37'): 0.040026,
    (33, 'feature_38'): -0.166606,
    (33, 'feature_39'): -1.022179,
    (33, 'feature_40'): -0.304155,
    (33, 'feature_41'): 0.003726,
    (33, 'feature_42'): -2.409069,
    (33, 'feature_43'): -0.582039,
    (33, 'feature_44'): -1.831875,
    (33, 'feature_45'): -1.298833,
    (33, 'feature_46'): -0.780279,
    (33, 'feature_47'): -3.018276,
    (33, 'feature_48'): -2.040749,
    (33, 'feature_49'): -4.439281,
    (33, 'feature_50'): -0.218588,
    (33, 'feature_51'): -0.204749,
    (33, 'feature_52'): 0.139378,
    (33, 'feature_53'): -2.207297,
    (33, 'feature_54'): -1.037298,
    (33, 'feature_55'): -3.012374,
    (33, 'feature_56'): -0.919469,
    (33, 'feature_57'): -1.390358,
    (33, 'feature_58'): -3.565477,
    (33, 'feature_59'): -1.244155,
    (33, 'feature_60'): -5.576054,
    (33, 'feature_61'): 0.845812,
    (33, 'feature_62'): -0.385368,
    (33, 'feature_63'): -0.226999,
    (33, 'feature_64'): -0.322854,
    (33, 'feature_65'): 0.190390,
    (33, 'feature_66'): 1.933878,
    (33, 'feature_67'): 0.407587,
    (33, 'feature_68'): 1.508223,
    (33, 'feature_69'): 1.879697,
    (33, 'feature_70'): 0.424723,
    (33, 'feature_71'): 0.919121,
    (33, 'feature_72'): 1.619425,
    (33, 'feature_73'): 0.057715,
    (33, 'feature_74'): 0.074882,
    (33, 'feature_75'): 3.389169,
    (33, 'feature_76'): 1.841311,
    (33, 'feature_77'): 0.567218,
    (33, 'feature_78'): 0.558530,
    (34, 'feature_00'): 0.113540,
    (34, 'feature_01'): -1.506321,
    (34, 'feature_02'): 0.287889,
    (34, 'feature_03'): 1.114241,
    (34, 'feature_04'): -0.619048,
    (34, 'feature_05'): -2.316985,
    (34, 'feature_06'): -0.342713,
    (34, 'feature_07'): -1.570461,
    (34, 'feature_08'): -0.053704,
    (34, 'feature_09'): 42.000000,
    (34, 'feature_10'): 5.000000,
    (34, 'feature_11'): 150.000000,
    (34, 'feature_12'): 0.112611,
    (34, 'feature_13'): -0.177417,
    (34, 'feature_14'): 0.178268,
    (34, 'feature_15'): -0.735336,
    (34, 'feature_16'): -0.331745,
    (34, 'feature_17'): -0.633386,
    (34, 'feature_18'): 0.249368,
    (34, 'feature_19'): 1.653411,
    (34, 'feature_20'): 0.002760,
    (34, 'feature_21'): -0.257053,
    (34, 'feature_22'): 1.166857,
    (34, 'feature_23'): 1.184520,
    (34, 'feature_24'): -0.362363,
    (34, 'feature_25'): -0.943673,
    (34, 'feature_26'): -0.176498,
    (34, 'feature_27'): 1.246821,
    (34, 'feature_28'): 1.283275,
    (34, 'feature_29'): -0.558301,
    (34, 'feature_30'): -0.840702,
    (34, 'feature_31'): -0.292120,
    (34, 'feature_32'): 0.863269,
    (34, 'feature_33'): 0.248571,
    (34, 'feature_34'): 1.312299,
    (34, 'feature_35'): 1.214976,
    (34, 'feature_36'): 0.421241,
    (34, 'feature_37'): -0.019187,
    (34, 'feature_38'): -0.135242,
    (34, 'feature_39'): 0.143081,
    (34, 'feature_40'): -0.533547,
    (34, 'feature_41'): -0.740368,
    (34, 'feature_42'): -1.387800,
    (34, 'feature_43'): -0.644368,
    (34, 'feature_44'): -1.282065,
    (34, 'feature_45'): -0.574014,
    (34, 'feature_46'): -1.390933,
    (34, 'feature_47'): -1.706069,
    (34, 'feature_48'): -0.821059,
    (34, 'feature_49'): -0.472650,
    (34, 'feature_50'): 0.222319,
    (34, 'feature_51'): -0.854981,
    (34, 'feature_52'): -1.732906,
    (34, 'feature_53'): -1.260130,
    (34, 'feature_54'): -0.744133,
    (34, 'feature_55'): -1.691536,
    (34, 'feature_56'): -0.865361,
    (34, 'feature_57'): -1.340579,
    (34, 'feature_58'): -1.953854,
    (34, 'feature_59'): -0.867560,
    (34, 'feature_60'): -0.797192,
    (34, 'feature_61'): 0.845812,
    (34, 'feature_62'): -0.491741,
    (34, 'feature_63'): -0.350290,
    (34, 'feature_64'): -0.271275,
    (34, 'feature_65'): 0.192680,
    (34, 'feature_66'): 1.365110,
    (34, 'feature_67'): 0.433228,
    (34, 'feature_68'): -0.092753,
    (34, 'feature_69'): 0.314330,
    (34, 'feature_70'): -0.202983,
    (34, 'feature_71'): -0.179797,
    (34, 'feature_72'): -0.053907,
    (34, 'feature_73'): -0.299687,
    (34, 'feature_74'): -0.202872,
    (34, 'feature_75'): -0.002706,
    (34, 'feature_76'): 0.006981,
    (34, 'feature_77'): -0.075956,
    (34, 'feature_78'): -0.061255,
    (35, 'feature_00'): 0.679088,
    (35, 'feature_01'): -1.118741,
    (35, 'feature_02'): 0.437552,
    (35, 'feature_03'): 0.521156,
    (35, 'feature_04'): -1.082304,
    (35, 'feature_05'): -1.589045,
    (35, 'feature_06'): -0.256223,
    (35, 'feature_07'): -1.306004,
    (35, 'feature_08'): -0.123386,
    (35, 'feature_09'): 25.000000,
    (35, 'feature_10'): 7.000000,
    (35, 'feature_11'): 195.000000,
    (35, 'feature_12'): -0.390751,
    (35, 'feature_13'): 0.058826,
    (35, 'feature_14'): 0.028477,
    (35, 'feature_15'): -0.510665,
    (35, 'feature_16'): -0.516720,
    (35, 'feature_17'): -0.353318,
    (35, 'feature_18'): 1.854869,
    (35, 'feature_19'): 0.714701,
    (35, 'feature_20'): 0.542222,
    (35, 'feature_21'): -0.084686,
    (35, 'feature_22'): -0.630074,
    (35, 'feature_23'): 0.033593,
    (35, 'feature_24'): 2.084893,
    (35, 'feature_25'): 1.046571,
    (35, 'feature_26'): 0.109437,
    (35, 'feature_27'): -1.286669,
    (35, 'feature_28'): -1.625050,
    (35, 'feature_29'): -0.216285,
    (35, 'feature_30'): -0.572598,
    (35, 'feature_31'): -0.099471,
    (35, 'feature_32'): 1.598203,
    (35, 'feature_33'): 0.740797,
    (35, 'feature_34'): 0.489917,
    (35, 'feature_35'): 1.509605,
    (35, 'feature_36'): 1.197052,
    (35, 'feature_37'): 0.320810,
    (35, 'feature_38'): 0.358670,
    (35, 'feature_39'): -0.301272,
    (35, 'feature_40'): 0.516723,
    (35, 'feature_41'): -0.236552,
    (35, 'feature_42'): -0.350861,
    (35, 'feature_43'): -0.223722,
    (35, 'feature_44'): -0.690084,
    (35, 'feature_45'): 0.207996,
    (35, 'feature_46'): -1.021887,
    (35, 'feature_47'): -0.417823,
    (35, 'feature_48'): -0.402063,
    (35, 'feature_49'): -0.356577,
    (35, 'feature_50'): 0.331667,
    (35, 'feature_51'): -0.356391,
    (35, 'feature_52'): -1.630592,
    (35, 'feature_53'): -0.720273,
    (35, 'feature_54'): -0.181455,
    (35, 'feature_55'): -1.650029,
    (35, 'feature_56'): -0.322848,
    (35, 'feature_57'): -0.791864,
    (35, 'feature_58'): -0.512363,
    (35, 'feature_59'): -0.901795,
    (35, 'feature_60'): -0.422817,
    (35, 'feature_61'): 0.845812,
    (35, 'feature_62'): -0.362909,
    (35, 'feature_63'): -0.128486,
    (35, 'feature_64'): -0.270118,
    (35, 'feature_65'): 1.535586,
    (35, 'feature_66'): 0.542281,
    (35, 'feature_67'): -0.224526,
    (35, 'feature_68'): 0.312888,
    (35, 'feature_69'): 0.119974,
    (35, 'feature_70'): -0.236240,
    (35, 'feature_71'): -0.220552,
    (35, 'feature_72'): -0.052969,
    (35, 'feature_73'): 0.079494,
    (35, 'feature_74'): 0.016070,
    (35, 'feature_75'): 0.371137,
    (35, 'feature_76'): 0.331724,
    (35, 'feature_77'): 0.088955,
    (35, 'feature_78'): 0.095219,
    (36, 'feature_00'): 0.042559,
    (36, 'feature_01'): -1.748191,
    (36, 'feature_02'): 0.211597,
    (36, 'feature_03'): 0.713040,
    (36, 'feature_04'): -0.983708,
    (36, 'feature_05'): -1.405621,
    (36, 'feature_06'): -0.227169,
    (36, 'feature_07'): -1.255891,
    (36, 'feature_08'): -0.110412,
    (36, 'feature_09'): 49.000000,
    (36, 'feature_10'): 7.000000,
    (36, 'feature_11'): 297.000000,
    (36, 'feature_12'): -0.428229,
    (36, 'feature_13'): -0.280789,
    (36, 'feature_14'): -0.269204,
    (36, 'feature_15'): -0.583605,
    (36, 'feature_16'): -0.643651,
    (36, 'feature_17'): -0.659119,
    (36, 'feature_18'): -0.163712,
    (36, 'feature_19'): 2.517011,
    (36, 'feature_20'): -0.108909,
    (36, 'feature_21'): 0.089721,
    (36, 'feature_22'): -0.897164,
    (36, 'feature_23'): -0.300418,
    (36, 'feature_24'): 1.143957,
    (36, 'feature_25'): 0.205031,
    (36, 'feature_26'): -0.367174,
    (36, 'feature_27'): -1.014297,
    (36, 'feature_28'): -1.477263,
    (36, 'feature_29'): -0.447969,
    (36, 'feature_30'): -0.503594,
    (36, 'feature_31'): 0.079240,
    (36, 'feature_32'): 1.142567,
    (36, 'feature_33'): -0.250004,
    (36, 'feature_34'): 1.226455,
    (36, 'feature_35'): 1.423985,
    (36, 'feature_36'): 0.218755,
    (36, 'feature_37'): -0.629031,
    (36, 'feature_38'): -0.287319,
    (36, 'feature_39'): 0.101322,
    (36, 'feature_40'): -0.539646,
    (36, 'feature_41'): 0.331670,
    (36, 'feature_42'): -0.912853,
    (36, 'feature_43'): -0.586665,
    (36, 'feature_44'): -0.166528,
    (36, 'feature_45'): 0.837126,
    (36, 'feature_46'): 0.581024,
    (36, 'feature_47'): -0.476562,
    (36, 'feature_48'): -0.625803,
    (36, 'feature_49'): -0.334692,
    (36, 'feature_50'): -0.595127,
    (36, 'feature_51'): -0.541823,
    (36, 'feature_52'): 0.615860,
    (36, 'feature_53'): -1.346922,
    (36, 'feature_54'): -0.072313,
    (36, 'feature_55'): -0.407001,
    (36, 'feature_56'): 0.706010,
    (36, 'feature_57'): -0.331250,
    (36, 'feature_58'): -0.527262,
    (36, 'feature_59'): -0.200182,
    (36, 'feature_60'): -0.528069,
    (36, 'feature_61'): 0.845812,
    (36, 'feature_62'): -0.498291,
    (36, 'feature_63'): -0.387475,
    (36, 'feature_64'): -0.217359,
    (36, 'feature_65'): 0.007208,
    (36, 'feature_66'): 2.756776,
    (36, 'feature_67'): -0.305334,
    (36, 'feature_68'): -0.240161,
    (36, 'feature_69'): -0.063571,
    (36, 'feature_70'): -0.451455,
    (36, 'feature_71'): -0.230157,
    (36, 'feature_72'): -0.356269,
    (36, 'feature_73'): -0.041393,
    (36, 'feature_74'): 0.034814,
    (36, 'feature_75'): -0.258361,
    (36, 'feature_76'): -0.171003,
    (36, 'feature_77'): 0.043774,
    (36, 'feature_78'): 0.126688,
    (37, 'feature_00'): 0.014789,
    (37, 'feature_01'): -1.961609,
    (37, 'feature_02'): 0.352037,
    (37, 'feature_03'): 0.816399,
    (37, 'feature_04'): -0.905738,
    (37, 'feature_05'): -1.192283,
    (37, 'feature_06'): -0.327741,
    (37, 'feature_07'): -1.812864,
    (37, 'feature_08'): -0.193394,
    (37, 'feature_09'): 34.000000,
    (37, 'feature_10'): 4.000000,
    (37, 'feature_11'): 214.000000,
    (37, 'feature_12'): -0.185520,
    (37, 'feature_13'): 0.354104,
    (37, 'feature_14'): 0.091549,
    (37, 'feature_15'): -0.642485,
    (37, 'feature_16'): -0.346708,
    (37, 'feature_17'): -0.610686,
    (37, 'feature_18'): -0.024380,
    (37, 'feature_19'): 2.603370,
    (37, 'feature_20'): -0.580293,
    (37, 'feature_21'): -0.242773,
    (37, 'feature_22'): -0.421886,
    (37, 'feature_23'): -0.214082,
    (37, 'feature_24'): 0.396354,
    (37, 'feature_25'): -0.958461,
    (37, 'feature_26'): 0.519482,
    (37, 'feature_27'): -0.455202,
    (37, 'feature_28'): -0.668232,
    (37, 'feature_29'): -0.597619,
    (37, 'feature_30'): -0.604933,
    (37, 'feature_31'): -0.119973,
    (37, 'feature_32'): 1.292358,
    (37, 'feature_33'): 0.201279,
    (37, 'feature_34'): 1.039528,
    (37, 'feature_35'): 1.138293,
    (37, 'feature_36'): 0.111943,
    (37, 'feature_37'): -0.142736,
    (37, 'feature_38'): -0.346362,
    (37, 'feature_39'): 0.119696,
    (37, 'feature_40'): 0.640974,
    (37, 'feature_41'): 0.646920,
    (37, 'feature_42'): -0.360532,
    (37, 'feature_43'): 0.095113,
    (37, 'feature_44'): 0.170844,
    (37, 'feature_45'): 1.095095,
    (37, 'feature_46'): 0.806499,
    (37, 'feature_47'): 0.497883,
    (37, 'feature_48'): 0.399707,
    (37, 'feature_49'): 0.419843,
    (37, 'feature_50'): 0.136760,
    (37, 'feature_51'): 0.073017,
    (37, 'feature_52'): 1.243543,
    (37, 'feature_53'): -0.147735,
    (37, 'feature_54'): -0.159892,
    (37, 'feature_55'): -1.014315,
    (37, 'feature_56'): 1.239644,
    (37, 'feature_57'): 0.397555,
    (37, 'feature_58'): 0.480661,
    (37, 'feature_59'): 0.121535,
    (37, 'feature_60'): 0.399882,
    (37, 'feature_61'): 0.845812,
    (37, 'feature_62'): -0.423643,
    (37, 'feature_63'): -0.195530,
    (37, 'feature_64'): -0.316500,
    (37, 'feature_65'): -0.407634,
    (37, 'feature_66'): 2.309432,
    (37, 'feature_67'): -0.190548,
    (37, 'feature_68'): 0.265017,
    (37, 'feature_69'): 0.154364,
    (37, 'feature_70'): -0.131415,
    (37, 'feature_71'): 0.309319,
    (37, 'feature_72'): 0.007948,
    (37, 'feature_73'): 0.431053,
    (37, 'feature_74'): 0.322892,
    (37, 'feature_75'): 1.565560,
    (37, 'feature_76'): 2.066381,
    (37, 'feature_77'): 0.710083,
    (37, 'feature_78'): 0.698162,
    (38, 'feature_00'): 0.176742,
    (38, 'feature_01'): -1.467811,
    (38, 'feature_02'): 0.822198,
    (38, 'feature_03'): -0.181564,
    (38, 'feature_04'): -0.807722,
    (38, 'feature_05'): -1.838525,
    (38, 'feature_06'): -0.297598,
    (38, 'feature_07'): -1.095958,
    (38, 'feature_08'): -0.132146,
    (38, 'feature_09'): 50.000000,
    (38, 'feature_10'): 1.000000,
    (38, 'feature_11'): 522.000000,
    (38, 'feature_12'): 0.619401,
    (38, 'feature_13'): 0.096979,
    (38, 'feature_14'): 0.549759,
    (38, 'feature_15'): -0.763634,
    (38, 'feature_16'): -0.458676,
    (38, 'feature_17'): -0.919894,
    (38, 'feature_18'): -2.847622,
    (38, 'feature_19'): 2.650321,
    (38, 'feature_20'): -0.346685,
    (38, 'feature_21'): -0.109926,
    (38, 'feature_22'): 1.264114,
    (38, 'feature_23'): 0.813752,
    (38, 'feature_24'): 0.745669,
    (38, 'feature_25'): -0.173604,
    (38, 'feature_26'): 0.394851,
    (38, 'feature_27'): 0.597030,
    (38, 'feature_28'): 0.015387,
    (38, 'feature_29'): -0.713974,
    (38, 'feature_30'): -0.729069,
    (38, 'feature_31'): -0.117773,
    (38, 'feature_32'): 1.295936,
    (38, 'feature_33'): -0.517270,
    (38, 'feature_34'): 1.337012,
    (38, 'feature_35'): 0.846260,
    (38, 'feature_36'): -1.336178,
    (38, 'feature_37'): -0.985287,
    (38, 'feature_38'): -1.318583,
    (38, 'feature_39'): -0.209264,
    (38, 'feature_40'): -0.244748,
    (38, 'feature_41'): 0.468189,
    (38, 'feature_42'): -0.375609,
    (38, 'feature_43'): -0.449057,
    (38, 'feature_44'): -0.367475,
    (38, 'feature_45'): 0.443066,
    (38, 'feature_46'): -0.137041,
    (38, 'feature_47'): -0.574692,
    (38, 'feature_48'): 0.314514,
    (38, 'feature_49'): -0.519814,
    (38, 'feature_50'): -0.571688,
    (38, 'feature_51'): 0.086809,
    (38, 'feature_52'): -0.048293,
    (38, 'feature_53'): -0.800866,
    (38, 'feature_54'): -0.253087,
    (38, 'feature_55'): -0.194572,
    (38, 'feature_56'): 1.747492,
    (38, 'feature_57'): 0.431749,
    (38, 'feature_58'): -0.736175,
    (38, 'feature_59'): 0.040388,
    (38, 'feature_60'): -0.425000,
    (38, 'feature_61'): 0.845812,
    (38, 'feature_62'): -0.378500,
    (38, 'feature_63'): -0.415880,
    (38, 'feature_64'): -0.425798,
    (38, 'feature_65'): -0.553193,
    (38, 'feature_66'): 2.021812,
    (38, 'feature_67'): 0.679037,
    (38, 'feature_68'): 0.074622,
    (38, 'feature_69'): 0.568490,
    (38, 'feature_70'): 0.315053,
    (38, 'feature_71'): 0.072298,
    (38, 'feature_72'): 0.196872,
    (38, 'feature_73'): -0.264101,
    (38, 'feature_74'): -0.061993,
    (38, 'feature_75'): 1.045759,
    (38, 'feature_76'): 1.517300,
    (38, 'feature_77'): 0.202212,
    (38, 'feature_78'): 0.444961,
}


#### Global constants for feature processing

In [66]:
FEATURE_COLS = [f'feature_{i:02d}' for i in range(79)]
FEATURE_NULL_COLS = [f'{col}_is_null' for col in FEATURE_COLS]
MODEL_COLS = ['symbol_id', 'weight'] + FEATURE_COLS + FEATURE_NULL_COLS
FEATURE_TYPES = {
    # Int16 features
    'feature_11': pl.Int16,
    
    # Int8 features
    'feature_09': pl.Int8,
    'feature_10': pl.Int8,
    
    # All other features are Float32
    **{f'feature_{i:02d}': pl.Float32 for i in range(79) 
       if i not in [9, 10, 11]}
}

In [67]:

def preprocess_features(df: pl.DataFrame) -> np.ndarray:
    """Preprocess features and return numpy array ready for prediction"""
    global last_known_values
    
    # Extract all needed data as numpy arrays once
    symbol_ids = df['symbol_id'].to_numpy()
    weights = df['weight'].to_numpy()
    
    # Preallocate final array (features + null flags + weight + symbol_id)
    n_rows = len(df)
    processed = np.zeros((n_rows, len(MODEL_COLS)), dtype=np.float32)
    
    # Fill symbol_id and weight columns first
    processed[:, 0] = symbol_ids
    processed[:, 1] = weights
    
    # Process features and create null flags
    for i, col in enumerate(FEATURE_COLS):
        values = df[col].to_numpy()
        null_mask = np.isnan(values)
        
        # Fill null values
        if null_mask.any():
            for idx in np.where(null_mask)[0]:
                sym = symbol_ids[idx]
                values[idx] = last_known_values.get((sym, col), 0.0)
        
        # Update last known values
        for sym, val in zip(symbol_ids[~null_mask], values[~null_mask]):
            last_known_values[(sym, col)] = val
            
        # Set feature values and null flags
        processed[:, 2+i] = values  # Features come after symbol_id and weight
        processed[:, 2+len(FEATURE_COLS)+i] = null_mask  # Null flags at the end
    
    return processed

### Load model

In [68]:
import lightgbm

In [69]:
# My models
# "val_r2_0.0122524.joblib"
# "backup_best_valr2.joblib"

# Benchmark Models
# LGBMV1_1.joblib

model_name = "backup_best_valr2.joblib"

In [70]:
def setup_prediction():
    """Load and setup model and preprocessing"""
    if IS_KAGGLE:
        model_path = trained_models_path + model_name
    else:
        model_path = os.path.join(PROJ_DIR, "training", "trained_models", model_name)
    
    # Load model directly as a Booster
    model = lightgbm.Booster(model_file=model_path)
    
    return model

# Load model globally
model = setup_prediction()

### Predict API

In [71]:
# TODO: FLoat value in predictions? Type 64?

# Global variables
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    
    if lags is not None:
        lags_ = lags

        # Update progress bar for new day if running locally
        if not IS_KAGGLE:
            progress_bar.update(1)

    # Preprocess and select needed columns
    processed = preprocess_features(test)
    
    # Convert to numpy and predict
    predictions = model.predict(processed)
    
    # Create return DataFrame efficiently
    predictions = test.select('row_id').with_columns(
        pl.Series('responder_6', predictions, dtype=pl.Float64)
    )
    
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame) # The predict function must return a DataFrame
    assert predictions.columns == ['row_id', 'responder_6']     # with columns 'row_id', 'responer_6'
    assert len(predictions) == len(test)                        # and as many rows as the test data.

    return predictions

### Submit

In [72]:
# Set up inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Run based on environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:    
    inference_server.run_local_gateway(
        (
            os.path.join(data_path, "test.parquet"),
            os.path.join(data_path, "lags.parquet"),
        )
    )

    # Close progress bar if it exists
    if progress_bar is not None:
        progress_bar.close()
    
    # Local scoring
    print("Scoring submission...")
    predictions = pl.read_parquet("submission.parquet")                      # Read submission predictions
    test_data = pl.read_parquet(os.path.join(data_path, "test.parquet"))     # Read test data with actual values

    # Score only rows marked for scoring
    mask = test_data['is_scored']
    score = r2_score_weighted(
        test_data.filter(mask)['responder_6'].to_numpy(),
        predictions.filter(mask)['responder_6'].to_numpy(),
        test_data.filter(mask)['weight'].to_numpy()
    )
    print(f"Overall R² score: {score}")

Processing days: 100%|██████████| 21/21 [04:21<00:00, 12.44s/it]

Scoring submission...
Overall R² score: -0.007271576846877181
